In [53]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

import scipy.stats as stats

#%matplotlib inline

In [54]:
example_datafile = 'example_data.txt'
filter_config = '../cpp/tests/assembly/connectivity.txt'

fed = open(example_datafile, 'r')
fc = open(filter_config, 'r')

In [55]:
fed = open(example_datafile, 'r')

begin_data = False
dns_data = {}
for line in fed.readlines():
    
    if begin_data:
        
        if "t = " in line:
            tc = float(line[3:])
            dns_data.update({tc:np.empty((0, 3))})
        else:
            data = [s.replace(',', '') for s in line.split()]
            pos = np.array([float(data[i]) for i in range(2, 5)])
            dns_data[tc] = np.vstack([dns_data[tc], pos])
    
    if (line.strip() == "BEGIN DATA"):
        begin_data = True
    
fed.close()

In [56]:
fc = open(filter_config, 'r')

begin_data = False

nodes = {}
conn = {}

for line in fc.readlines():
    
    if begin_data:
    
        data = [s.replace(',', '') for s in line.split()]
        
        if(len(data)>0):
        
            if (data[0]=='N'):
                nid = int(data[1])
                pos = np.array([float(data[i]) for i in range(2, len(data))])
                nodes.update({nid:pos})
                
            elif (data[0]=='E'):
                eid = int(data[2])
                nids = [int(data[i]) for i in range(3, len(data))]
                conn.update({eid:nids})
        
    if (line.strip() == "BEGIN DATA"):
        begin_data = True

print(nodes)
print(conn)
fc.close()

{1: array([0., 0., 0.]), 2: array([1., 0., 0.]), 3: array([1., 1., 0.]), 4: array([0., 1., 0.]), 5: array([0., 0., 1.]), 8: array([1., 0., 1.]), 6: array([1., 1., 1.]), 7: array([0., 1., 1.])}
{10: [1, 2, 3, 4, 5, 8, 6, 7]}


In [57]:
#Assume hex8 for now
def plot_hex(eid, nodes, conn, u, ax):
    #Plot bottom square
    x = np.array([nodes[n]+u[n] for n in conn[eid][:4]] + [nodes[conn[eid][0]]+u[conn[eid][0]]])
    ax.plot(*zip(*x), color='k', marker='o')
    #Plot top square
    x = np.array([nodes[n]+u[n] for n in conn[eid][4:]] + [nodes[conn[eid][4]]+u[conn[eid][4]]])
    ax.plot(*zip(*x), color='k', marker='o')
    #Plot sides
    for i in range(4):
        x = [nodes[conn[eid][i]]+u[conn[eid][i]], nodes[conn[eid][i+4]]+u[conn[eid][i+4]]]
        ax.plot(*zip(*x), color='k', marker='o')

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(111, projection='3d')
u = dict((key, np.zeros(3)) for key in nodes.keys())
plot_hex(10, nodes, conn, u, ax)
plt.show()

In [58]:
us = np.array([[0.296511, -0.100599, 0.288297],\
[0.340768, -0.0339587, 0.321454],\
[0.297521, -0.0599225, 0.386237],\
[0.253264, -0.126563, 0.35308],\
[0.273059, -0.189617, 0.274714],\
[0.317316, -0.122977, 0.307871],\
[0.274069, -0.148941, 0.372654],\
[0.229812, -0.215581, 0.339497]])

u = dict([(key, us[i]) for key, i in zip(nodes.keys(), range(8))])

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(111, projection='3d')
for i, key in enumerate(dns_data.keys()):
    if (i != 1):
        continue
    else:
        print(key)
    data = dns_data[key]
    ax.scatter(*zip(*data), label='t = {0:1.4f}'.format(key))
plot_hex(10, nodes, conn, u, ax)
plt.show()

0.1


In [59]:
us = np.array([[0.602945, -0.204222, 0.561868],
[0.679657, -0.0624096, 0.6374],
[0.592969, -0.126602, 0.760303],
[0.516257, -0.268414, 0.684771],
[0.57752, -0.381413, 0.51808],
[0.654232, -0.239601, 0.593612],
[0.567544, -0.303793, 0.716514],
[0.490832, -0.445605, 0.640982]])



u = dict([(key, us[i]) for key, i in zip(nodes.keys(), range(8))])
print(u)

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(111, projection='3d')
for i, key in enumerate(dns_data.keys()):
    if (i != 2):
        continue
    data = dns_data[key]
    ax.scatter(*zip(*data), label='t = {0:1.4f}'.format(key))
plot_hex(10, nodes, conn, u, ax)

{1: array([ 0.602945, -0.204222,  0.561868]), 2: array([ 0.679657 , -0.0624096,  0.6374   ]), 3: array([ 0.592969, -0.126602,  0.760303]), 4: array([ 0.516257, -0.268414,  0.684771]), 5: array([ 0.57752 , -0.381413,  0.51808 ]), 8: array([ 0.654232, -0.239601,  0.593612]), 6: array([ 0.567544, -0.303793,  0.716514]), 7: array([ 0.490832, -0.445605,  0.640982])}


In [60]:
[1,2,3,]

[1, 2, 3]

In [61]:
micro_us = np.array([[0.32, -0.05, 0.2, ],
[0.314714, -0.0700651, 0.196938, ],
[0.309428, -0.0901302, 0.193877, ],
[0.304142, -0.110195, 0.190815, ],
[0.298855, -0.13026, 0.187754, ],
[0.293569, -0.150326, 0.184692, ],
[0.288283, -0.170391, 0.18163, ],
[0.282997, -0.190456, 0.178569, ],
[0.277711, -0.210521, 0.175507, ],
[0.272425, -0.230586, 0.172446, ],
[0.329976, -0.0349789, 0.207474, ],
[0.32469, -0.055044, 0.204412, ],
[0.319403, -0.0751092, 0.20135, ],
[0.314117, -0.0951743, 0.198289, ],
[0.308831, -0.115239, 0.195227, ],
[0.303545, -0.135305, 0.192166, ],
[0.298259, -0.15537, 0.189104, ],
[0.292973, -0.175435, 0.186042, ],
[0.287687, -0.1955, 0.182981, ],
[0.282401, -0.215565, 0.179919, ],
[0.339952, -0.0199578, 0.214947, ],
[0.334665, -0.040023, 0.211886, ],
[0.329379, -0.0600881, 0.208824, ],
[0.324093, -0.0801532, 0.205763, ],
[0.318807, -0.100218, 0.202701, ],
[0.313521, -0.120283, 0.199639, ],
[0.308235, -0.140349, 0.196578, ],
[0.302949, -0.160414, 0.193516, ],
[0.297662, -0.180479, 0.190455, ],
[0.292376, -0.200544, 0.187393, ],
[0.349927, -0.00493675, 0.222421, ],
[0.344641, -0.0250019, 0.21936, ],
[0.339355, -0.045067, 0.216298, ],
[0.334069, -0.0651321, 0.213236, ],
[0.328783, -0.0851973, 0.210175, ],
[0.323497, -0.105262, 0.207113, ],
[0.31821, -0.125328, 0.204051, ],
[0.312924, -0.145393, 0.20099, ],
[0.307638, -0.165458, 0.197928, ],
[0.302352, -0.185523, 0.194867, ],
[0.359903, 0.0100843, 0.229895, ],
[0.354617, -0.0099808, 0.226833, ],
[0.349331, -0.0300459, 0.223772, ],
[0.344045, -0.050111, 0.22071, ],
[0.338758, -0.0701762, 0.217648, ],
[0.333472, -0.0902413, 0.214587, ],
[0.328186, -0.110306, 0.211525, ],
[0.3229, -0.130372, 0.208464, ],
[0.317614, -0.150437, 0.205402, ],
[0.312328, -0.170502, 0.20234, ],
[0.369879, 0.0251054, 0.237369, ],
[0.364593, 0.00504029, 0.234307, ],
[0.359307, -0.0150248, 0.231245, ],
[0.35402, -0.03509, 0.228184, ],
[0.348734, -0.0551551, 0.225122, ],
[0.343448, -0.0752202, 0.22206, ],
[0.338162, -0.0952853, 0.218999, ],
[0.332876, -0.11535, 0.215937, ],
[0.32759, -0.135416, 0.212876, ],
[0.322304, -0.155481, 0.209814, ],
[0.379855, 0.0401265, 0.244842, ],
[0.374568, 0.0200614, 0.241781, ],
[0.369282, -3.75585e-06, 0.238719, ],
[0.363996, -0.0200689, 0.235657, ],
[0.35871, -0.040134, 0.232596, ],
[0.353424, -0.0601991, 0.229534, ],
[0.348138, -0.0802643, 0.226473, ],
[0.342852, -0.100329, 0.223411, ],
[0.337565, -0.120395, 0.220349, ],
[0.332279, -0.14046, 0.217288, ],
[0.38983, 0.0551476, 0.252316, ],
[0.384544, 0.0350825, 0.249254, ],
[0.379258, 0.0150173, 0.246193, ],
[0.373972, -0.0050478, 0.243131, ],
[0.368686, -0.0251129, 0.24007, ],
[0.3634, -0.045178, 0.237008, ],
[0.358113, -0.0652432, 0.233946, ],
[0.352827, -0.0853083, 0.230885, ],
[0.347541, -0.105373, 0.227823, ],
[0.342255, -0.125439, 0.224761, ],
[0.399806, 0.0701687, 0.25979, ],
[0.39452, 0.0501035, 0.256728, ],
[0.389234, 0.0300384, 0.253666, ],
[0.383948, 0.00997328, 0.250605, ],
[0.378662, -0.0100918, 0.247543, ],
[0.373375, -0.030157, 0.244482, ],
[0.368089, -0.0502221, 0.24142, ],
[0.362803, -0.0702872, 0.238358, ],
[0.357517, -0.0903523, 0.235297, ],
[0.352231, -0.110417, 0.232235, ],
[0.409782, 0.0851897, 0.267263, ],
[0.404496, 0.0651246, 0.264202, ],
[0.39921, 0.0450595, 0.26114, ],
[0.393923, 0.0249944, 0.258079, ],
[0.388637, 0.00492924, 0.255017, ],
[0.383351, -0.0151359, 0.251955, ],
[0.378065, -0.035201, 0.248894, ],
[0.372779, -0.0552661, 0.245832, ],
[0.367493, -0.0753313, 0.242771, ],
[0.362207, -0.0953964, 0.239709, ],
[0.310252, -0.0558524, 0.214602, ],
[0.304966, -0.0759175, 0.211541, ],
[0.29968, -0.0959826, 0.208479, ],
[0.294394, -0.116048, 0.205418, ],
[0.289107, -0.136113, 0.202356, ],
[0.283821, -0.156178, 0.199294, ],
[0.278535, -0.176243, 0.196233, ],
[0.273249, -0.196308, 0.193171, ],
[0.267963, -0.216373, 0.19011, ],
[0.262677, -0.236439, 0.187048, ],
[0.320228, -0.0408313, 0.222076, ],
[0.314942, -0.0608964, 0.219015, ],
[0.309655, -0.0809616, 0.215953, ],
[0.304369, -0.101027, 0.212891, ],
[0.299083, -0.121092, 0.20983, ],
[0.293797, -0.141157, 0.206768, ],
[0.288511, -0.161222, 0.203706, ],
[0.283225, -0.181287, 0.200645, ],
[0.277939, -0.201352, 0.197583, ],
[0.272653, -0.221417, 0.194522, ],
[0.330203, -0.0258102, 0.22955, ],
[0.324917, -0.0458753, 0.226488, ],
[0.319631, -0.0659405, 0.223427, ],
[0.314345, -0.0860056, 0.220365, ],
[0.309059, -0.106071, 0.217303, ],
[0.303773, -0.126136, 0.214242, ],
[0.298487, -0.146201, 0.21118, ],
[0.293201, -0.166266, 0.208119, ],
[0.287914, -0.186331, 0.205057, ],
[0.282628, -0.206396, 0.201995, ],
[0.340179, -0.0107891, 0.237024, ],
[0.334893, -0.0308543, 0.233962, ],
[0.329607, -0.0509194, 0.2309, ],
[0.324321, -0.0709845, 0.227839, ],
[0.319035, -0.0910496, 0.224777, ],
[0.313749, -0.111115, 0.221715, ],
[0.308462, -0.13118, 0.218654, ],
[0.303176, -0.151245, 0.215592, ],
[0.29789, -0.17131, 0.212531, ],
[0.292604, -0.191375, 0.209469, ],
[0.350155, 0.00423195, 0.244497, ],
[0.344869, -0.0158332, 0.241436, ],
[0.339583, -0.0358983, 0.238374, ],
[0.334297, -0.0559634, 0.235312, ],
[0.32901, -0.0760286, 0.232251, ],
[0.323724, -0.0960937, 0.229189, ],
[0.318438, -0.116159, 0.226128, ],
[0.313152, -0.136224, 0.223066, ],
[0.307866, -0.156289, 0.220004, ],
[0.30258, -0.176354, 0.216943, ],
[0.360131, 0.019253, 0.251971, ],
[0.354845, -0.000812097, 0.248909, ],
[0.349558, -0.0208772, 0.245848, ],
[0.344272, -0.0409423, 0.242786, ],
[0.338986, -0.0610075, 0.239725, ],
[0.3337, -0.0810726, 0.236663, ],
[0.328414, -0.101138, 0.233601, ],
[0.323128, -0.121203, 0.23054, ],
[0.317842, -0.141268, 0.227478, ],
[0.312556, -0.161333, 0.224416, ],
[0.370107, 0.0342741, 0.259445, ],
[0.36482, 0.014209, 0.256383, ],
[0.359534, -0.00585614, 0.253321, ],
[0.354248, -0.0259213, 0.25026, ],
[0.348962, -0.0459864, 0.247198, ],
[0.343676, -0.0660515, 0.244137, ],
[0.33839, -0.0861166, 0.241075, ],
[0.333104, -0.106182, 0.238013, ],
[0.327817, -0.126247, 0.234952, ],
[0.322531, -0.146312, 0.23189, ],
[0.380082, 0.0492952, 0.266918, ],
[0.374796, 0.0292301, 0.263857, ],
[0.36951, 0.00916494, 0.260795, ],
[0.364224, -0.0109002, 0.257734, ],
[0.358938, -0.0309653, 0.254672, ],
[0.353652, -0.0510304, 0.25161, ],
[0.348365, -0.0710956, 0.248549, ],
[0.343079, -0.0911607, 0.245487, ],
[0.337793, -0.111226, 0.242426, ],
[0.332507, -0.131291, 0.239364, ],
[0.390058, 0.0643163, 0.274392, ],
[0.384772, 0.0442511, 0.27133, ],
[0.379486, 0.024186, 0.268269, ],
[0.3742, 0.0041209, 0.265207, ],
[0.368913, -0.0159442, 0.262146, ],
[0.363627, -0.0360093, 0.259084, ],
[0.358341, -0.0560745, 0.256022, ],
[0.353055, -0.0761396, 0.252961, ],
[0.347769, -0.0962047, 0.249899, ],
[0.342483, -0.11627, 0.246838, ],
[0.400034, 0.0793374, 0.281866, ],
[0.394748, 0.0592722, 0.278804, ],
[0.389462, 0.0392071, 0.275743, ],
[0.384175, 0.019142, 0.272681, ],
[0.378889, -0.000923142, 0.269619, ],
[0.373603, -0.0209883, 0.266558, ],
[0.368317, -0.0410534, 0.263496, ],
[0.363031, -0.0611185, 0.260435, ],
[0.357745, -0.0811836, 0.257373, ],
[0.352459, -0.101249, 0.254311, ],
[0.300504, -0.0617048, 0.229205, ],
[0.295218, -0.0817699, 0.226143, ],
[0.289932, -0.101835, 0.223082, ],
[0.284646, -0.1219, 0.22002, ],
[0.279359, -0.141965, 0.216958, ],
[0.274073, -0.16203, 0.213897, ],
[0.268787, -0.182096, 0.210835, ],
[0.263501, -0.202161, 0.207774, ],
[0.258215, -0.222226, 0.204712, ],
[0.252929, -0.242291, 0.20165, ],
[0.31048, -0.0466837, 0.236679, ],
[0.305194, -0.0667488, 0.233617, ],
[0.299907, -0.0868139, 0.230555, ],
[0.294621, -0.106879, 0.227494, ],
[0.289335, -0.126944, 0.224432, ],
[0.284049, -0.147009, 0.22137, ],
[0.278763, -0.167074, 0.218309, ],
[0.273477, -0.18714, 0.215247, ],
[0.268191, -0.207205, 0.212186, ],
[0.262905, -0.22727, 0.209124, ],
[0.320455, -0.0316626, 0.244152, ],
[0.315169, -0.0517277, 0.241091, ],
[0.309883, -0.0717929, 0.238029, ],
[0.304597, -0.091858, 0.234967, ],
[0.299311, -0.111923, 0.231906, ],
[0.294025, -0.131988, 0.228844, ],
[0.288739, -0.152053, 0.225783, ],
[0.283453, -0.172118, 0.222721, ],
[0.278166, -0.192184, 0.219659, ],
[0.27288, -0.212249, 0.216598, ],
[0.330431, -0.0166415, 0.251626, ],
[0.325145, -0.0367066, 0.248564, ],
[0.319859, -0.0567718, 0.245503, ],
[0.314573, -0.0768369, 0.242441, ],
[0.309287, -0.096902, 0.23938, ],
[0.304001, -0.116967, 0.236318, ],
[0.298714, -0.137032, 0.233256, ],
[0.293428, -0.157097, 0.230195, ],
[0.288142, -0.177163, 0.227133, ],
[0.282856, -0.197228, 0.224071, ],
[0.340407, -0.00162044, 0.2591, ],
[0.335121, -0.0216856, 0.256038, ],
[0.329835, -0.0417507, 0.252976, ],
[0.324549, -0.0618158, 0.249915, ],
[0.319262, -0.0818809, 0.246853, ],
[0.313976, -0.101946, 0.243792, ],
[0.30869, -0.122011, 0.24073, ],
[0.303404, -0.142076, 0.237668, ],
[0.298118, -0.162141, 0.234607, ],
[0.292832, -0.182207, 0.231545, ],
[0.350383, 0.0134006, 0.266573, ],
[0.345097, -0.00666448, 0.263512, ],
[0.33981, -0.0267296, 0.26045, ],
[0.334524, -0.0467947, 0.257389, ],
[0.329238, -0.0668599, 0.254327, ],
[0.323952, -0.086925, 0.251265, ],
[0.318666, -0.10699, 0.248204, ],
[0.31338, -0.127055, 0.245142, ],
[0.308094, -0.14712, 0.242081, ],
[0.302808, -0.167185, 0.239019, ],
[0.360358, 0.0284217, 0.274047, ],
[0.355072, 0.0083566, 0.270985, ],
[0.349786, -0.0117085, 0.267924, ],
[0.3445, -0.0317736, 0.264862, ],
[0.339214, -0.0518388, 0.261801, ],
[0.333928, -0.0719039, 0.258739, ],
[0.328642, -0.091969, 0.255677, ],
[0.323356, -0.112034, 0.252616, ],
[0.318069, -0.132099, 0.249554, ],
[0.312783, -0.152164, 0.246493, ],
[0.370334, 0.0434428, 0.281521, ],
[0.365048, 0.0233777, 0.278459, ],
[0.359762, 0.00331256, 0.275398, ],
[0.354476, -0.0167526, 0.272336, ],
[0.34919, -0.0368177, 0.269274, ],
[0.343904, -0.0568828, 0.266213, ],
[0.338617, -0.0769479, 0.263151, ],
[0.333331, -0.0970131, 0.26009, ],
[0.328045, -0.117078, 0.257028, ],
[0.322759, -0.137143, 0.253966, ],
[0.38031, 0.0584639, 0.288995, ],
[0.375024, 0.0383988, 0.285933, ],
[0.369738, 0.0183336, 0.282871, ],
[0.364452, -0.00173148, 0.27981, ],
[0.359165, -0.0217966, 0.276748, ],
[0.353879, -0.0418617, 0.273686, ],
[0.348593, -0.0619269, 0.270625, ],
[0.343307, -0.081992, 0.267563, ],
[0.338021, -0.102057, 0.264502, ],
[0.332735, -0.122122, 0.26144, ],
[0.390286, 0.073485, 0.296468, ],
[0.385, 0.0534198, 0.293407, ],
[0.379713, 0.0333547, 0.290345, ],
[0.374427, 0.0132896, 0.287283, ],
[0.369141, -0.00677553, 0.284222, ],
[0.363855, -0.0268406, 0.28116, ],
[0.358569, -0.0469058, 0.278099, ],
[0.353283, -0.0669709, 0.275037, ],
[0.347997, -0.087036, 0.271975, ],
[0.342711, -0.107101, 0.268914, ],
[0.290756, -0.0675572, 0.243807, ],
[0.28547, -0.0876223, 0.240746, ],
[0.280184, -0.107687, 0.237684, ],
[0.274898, -0.127753, 0.234622, ],
[0.269611, -0.147818, 0.231561, ],
[0.264325, -0.167883, 0.228499, ],
[0.259039, -0.187948, 0.225438, ],
[0.253753, -0.208013, 0.222376, ],
[0.248467, -0.228078, 0.219314, ],
[0.243181, -0.248143, 0.216253, ],
[0.300732, -0.0525361, 0.251281, ],
[0.295446, -0.0726012, 0.248219, ],
[0.290159, -0.0926663, 0.245158, ],
[0.284873, -0.112731, 0.242096, ],
[0.279587, -0.132797, 0.239035, ],
[0.274301, -0.152862, 0.235973, ],
[0.269015, -0.172927, 0.232911, ],
[0.263729, -0.192992, 0.22985, ],
[0.258443, -0.213057, 0.226788, ],
[0.253156, -0.233122, 0.223726, ],
[0.310707, -0.037515, 0.258755, ],
[0.305421, -0.0575801, 0.255693, ],
[0.300135, -0.0776452, 0.252631, ],
[0.294849, -0.0977104, 0.24957, ],
[0.289563, -0.117775, 0.246508, ],
[0.284277, -0.137841, 0.243447, ],
[0.278991, -0.157906, 0.240385, ],
[0.273705, -0.177971, 0.237323, ],
[0.268418, -0.198036, 0.234262, ],
[0.263132, -0.218101, 0.2312, ],
[0.320683, -0.0224939, 0.266228, ],
[0.315397, -0.042559, 0.263167, ],
[0.310111, -0.0626242, 0.260105, ],
[0.304825, -0.0826893, 0.257044, ],
[0.299539, -0.102754, 0.253982, ],
[0.294253, -0.12282, 0.25092, ],
[0.288966, -0.142885, 0.247859, ],
[0.28368, -0.16295, 0.244797, ],
[0.278394, -0.183015, 0.241735, ],
[0.273108, -0.20308, 0.238674, ],
[0.330659, -0.00747282, 0.273702, ],
[0.325373, -0.0275379, 0.27064, ],
[0.320087, -0.0476031, 0.267579, ],
[0.314801, -0.0676682, 0.264517, ],
[0.309514, -0.0877333, 0.261456, ],
[0.304228, -0.107798, 0.258394, ],
[0.298942, -0.127864, 0.255332, ],
[0.293656, -0.147929, 0.252271, ],
[0.28837, -0.167994, 0.249209, ],
[0.283084, -0.188059, 0.246148, ],
[0.340635, 0.00754826, 0.281176, ],
[0.335349, -0.0125169, 0.278114, ],
[0.330062, -0.032582, 0.275053, ],
[0.324776, -0.0526471, 0.271991, ],
[0.31949, -0.0727122, 0.268929, ],
[0.314204, -0.0927774, 0.265868, ],
[0.308918, -0.112842, 0.262806, ],
[0.303632, -0.132908, 0.259745, ],
[0.298346, -0.152973, 0.256683, ],
[0.29306, -0.173038, 0.253621, ],
[0.35061, 0.0225693, 0.28865, ],
[0.345324, 0.00250422, 0.285588, ],
[0.340038, -0.0175609, 0.282526, ],
[0.334752, -0.037626, 0.279465, ],
[0.329466, -0.0576912, 0.276403, ],
[0.32418, -0.0777563, 0.273341, ],
[0.318894, -0.0978214, 0.27028, ],
[0.313608, -0.117887, 0.267218, ],
[0.308321, -0.137952, 0.264157, ],
[0.303035, -0.158017, 0.261095, ],
[0.360586, 0.0375904, 0.296123, ],
[0.3553, 0.0175253, 0.293062, ],
[0.350014, -0.00253982, 0.29, ],
[0.344728, -0.0226049, 0.286938, ],
[0.339442, -0.0426701, 0.283877, ],
[0.334156, -0.0627352, 0.280815, ],
[0.328869, -0.0828003, 0.277754, ],
[0.323583, -0.102865, 0.274692, ],
[0.318297, -0.122931, 0.27163, ],
[0.313011, -0.142996, 0.268569, ],
[0.370562, 0.0526115, 0.303597, ],
[0.365276, 0.0325464, 0.300535, ],
[0.35999, 0.0124813, 0.297474, ],
[0.354704, -0.00758387, 0.294412, ],
[0.349417, -0.027649, 0.29135, ],
[0.344131, -0.0477141, 0.288289, ],
[0.338845, -0.0677792, 0.285227, ],
[0.333559, -0.0878444, 0.282166, ],
[0.328273, -0.107909, 0.279104, ],
[0.322987, -0.127975, 0.276042, ],
[0.380538, 0.0676326, 0.311071, ],
[0.375252, 0.0475675, 0.308009, ],
[0.369965, 0.0275023, 0.304947, ],
[0.364679, 0.00743722, 0.301886, ],
[0.359393, -0.0126279, 0.298824, ],
[0.354107, -0.032693, 0.295763, ],
[0.348821, -0.0527582, 0.292701, ],
[0.343535, -0.0728233, 0.289639, ],
[0.338249, -0.0928884, 0.286578, ],
[0.332963, -0.112954, 0.283516, ],
[0.281008, -0.0734095, 0.25841, ],
[0.275722, -0.0934747, 0.255348, ],
[0.270436, -0.11354, 0.252286, ],
[0.26515, -0.133605, 0.249225, ],
[0.259863, -0.15367, 0.246163, ],
[0.254577, -0.173735, 0.243102, ],
[0.249291, -0.1938, 0.24004, ],
[0.244005, -0.213865, 0.236978, ],
[0.238719, -0.233931, 0.233917, ],
[0.233433, -0.253996, 0.230855, ],
[0.290984, -0.0583885, 0.265883, ],
[0.285698, -0.0784536, 0.262822, ],
[0.280411, -0.0985187, 0.25976, ],
[0.275125, -0.118584, 0.256699, ],
[0.269839, -0.138649, 0.253637, ],
[0.264553, -0.158714, 0.250575, ],
[0.259267, -0.178779, 0.247514, ],
[0.253981, -0.198844, 0.244452, ],
[0.248695, -0.218909, 0.24139, ],
[0.243408, -0.238975, 0.238329, ],
[0.300959, -0.0433674, 0.273357, ],
[0.295673, -0.0634325, 0.270295, ],
[0.290387, -0.0834976, 0.267234, ],
[0.285101, -0.103563, 0.264172, ],
[0.279815, -0.123628, 0.261111, ],
[0.274529, -0.143693, 0.258049, ],
[0.269243, -0.163758, 0.254987, ],
[0.263956, -0.183823, 0.251926, ],
[0.25867, -0.203888, 0.248864, ],
[0.253384, -0.223953, 0.245803, ],
[0.310935, -0.0283463, 0.280831, ],
[0.305649, -0.0484114, 0.277769, ],
[0.300363, -0.0684765, 0.274708, ],
[0.295077, -0.0885417, 0.271646, ],
[0.289791, -0.108607, 0.268584, ],
[0.284505, -0.128672, 0.265523, ],
[0.279218, -0.148737, 0.262461, ],
[0.273932, -0.168802, 0.2594, ],
[0.268646, -0.188867, 0.256338, ],
[0.26336, -0.208932, 0.253276, ],
[0.320911, -0.0133252, 0.288305, ],
[0.315625, -0.0333903, 0.285243, ],
[0.310339, -0.0534555, 0.282181, ],
[0.305053, -0.0735206, 0.27912, ],
[0.299766, -0.0935857, 0.276058, ],
[0.29448, -0.113651, 0.272996, ],
[0.289194, -0.133716, 0.269935, ],
[0.283908, -0.153781, 0.266873, ],
[0.278622, -0.173846, 0.263812, ],
[0.273336, -0.193911, 0.26075, ],
[0.330887, 0.00169588, 0.295778, ],
[0.325601, -0.0183692, 0.292717, ],
[0.320314, -0.0384344, 0.289655, ],
[0.315028, -0.0584995, 0.286593, ],
[0.309742, -0.0785646, 0.283532, ],
[0.304456, -0.0986297, 0.28047, ],
[0.29917, -0.118695, 0.277409, ],
[0.293884, -0.13876, 0.274347, ],
[0.288598, -0.158825, 0.271285, ],
[0.283311, -0.17889, 0.268224, ],
[0.340862, 0.016717, 0.303252, ],
[0.335576, -0.00334817, 0.30019, ],
[0.33029, -0.0234133, 0.297129, ],
[0.325004, -0.0434784, 0.294067, ],
[0.319718, -0.0635435, 0.291005, ],
[0.314432, -0.0836087, 0.287944, ],
[0.309146, -0.103674, 0.284882, ],
[0.30386, -0.123739, 0.281821, ],
[0.298573, -0.143804, 0.278759, ],
[0.293287, -0.163869, 0.275697, ],
[0.350838, 0.031738, 0.310726, ],
[0.345552, 0.0116729, 0.307664, ],
[0.340266, -0.00839221, 0.304602, ],
[0.33498, -0.0284573, 0.301541, ],
[0.329694, -0.0485225, 0.298479, ],
[0.324408, -0.0685876, 0.295418, ],
[0.319121, -0.0886527, 0.292356, ],
[0.313835, -0.108718, 0.289294, ],
[0.308549, -0.128783, 0.286233, ],
[0.303263, -0.148848, 0.283171, ],
[0.360814, 0.0467591, 0.318199, ],
[0.355528, 0.026694, 0.315138, ],
[0.350242, 0.00662887, 0.312076, ],
[0.344956, -0.0134363, 0.309015, ],
[0.339669, -0.0335014, 0.305953, ],
[0.334383, -0.0535665, 0.302891, ],
[0.329097, -0.0736316, 0.29983, ],
[0.323811, -0.0936967, 0.296768, ],
[0.318525, -0.113762, 0.293706, ],
[0.313239, -0.133827, 0.290645, ],
[0.37079, 0.0617802, 0.325673, ],
[0.365504, 0.0417151, 0.322611, ],
[0.360217, 0.02165, 0.31955, ],
[0.354931, 0.00158483, 0.316488, ],
[0.349645, -0.0184803, 0.313427, ],
[0.344359, -0.0385454, 0.310365, ],
[0.339073, -0.0586105, 0.307303, ],
[0.333787, -0.0786757, 0.304242, ],
[0.328501, -0.0987408, 0.30118, ],
[0.323214, -0.118806, 0.298119, ],
[0.27126, -0.0792619, 0.273012, ],
[0.265974, -0.099327, 0.26995, ],
[0.260688, -0.119392, 0.266889, ],
[0.255401, -0.139457, 0.263827, ],
[0.250115, -0.159522, 0.260766, ],
[0.244829, -0.179588, 0.257704, ],
[0.239543, -0.199653, 0.254642, ],
[0.234257, -0.219718, 0.251581, ],
[0.228971, -0.239783, 0.248519, ],
[0.223685, -0.259848, 0.245458, ],
[0.281236, -0.0642408, 0.280486, ],
[0.27595, -0.084306, 0.277424, ],
[0.270663, -0.104371, 0.274363, ],
[0.265377, -0.124436, 0.271301, ],
[0.260091, -0.144501, 0.268239, ],
[0.254805, -0.164566, 0.265178, ],
[0.249519, -0.184632, 0.262116, ],
[0.244233, -0.204697, 0.259055, ],
[0.238947, -0.224762, 0.255993, ],
[0.23366, -0.244827, 0.252931, ],
[0.291211, -0.0492198, 0.28796, ],
[0.285925, -0.0692849, 0.284898, ],
[0.280639, -0.08935, 0.281836, ],
[0.275353, -0.109415, 0.278775, ],
[0.270067, -0.12948, 0.275713, ],
[0.264781, -0.149545, 0.272651, ],
[0.259495, -0.169611, 0.26959, ],
[0.254208, -0.189676, 0.266528, ],
[0.248922, -0.209741, 0.263467, ],
[0.243636, -0.229806, 0.260405, ],
[0.301187, -0.0341987, 0.295433, ],
[0.295901, -0.0542638, 0.292372, ],
[0.290615, -0.0743289, 0.28931, ],
[0.285329, -0.094394, 0.286248, ],
[0.280043, -0.114459, 0.283187, ],
[0.274756, -0.134524, 0.280125, ],
[0.26947, -0.154589, 0.277064, ],
[0.264184, -0.174655, 0.274002, ],
[0.258898, -0.19472, 0.27094, ],
[0.253612, -0.214785, 0.267879, ],
[0.311163, -0.0191776, 0.302907, ],
[0.305877, -0.0392427, 0.299845, ],
[0.300591, -0.0593078, 0.296784, ],
[0.295305, -0.079373, 0.293722, ],
[0.290018, -0.0994381, 0.29066, ],
[0.284732, -0.119503, 0.287599, ],
[0.279446, -0.139568, 0.284537, ],
[0.27416, -0.159633, 0.281476, ],
[0.268874, -0.179699, 0.278414, ],
[0.263588, -0.199764, 0.275352, ],
[0.321139, -0.00415651, 0.310381, ],
[0.315853, -0.0242216, 0.307319, ],
[0.310566, -0.0442868, 0.304257, ],
[0.30528, -0.0643519, 0.301196, ],
[0.299994, -0.084417, 0.298134, ],
[0.294708, -0.104482, 0.295073, ],
[0.289422, -0.124547, 0.292011, ],
[0.284136, -0.144612, 0.288949, ],
[0.27885, -0.164678, 0.285888, ],
[0.273563, -0.184743, 0.282826, ],
[0.331114, 0.0108646, 0.317854, ],
[0.325828, -0.00920055, 0.314793, ],
[0.320542, -0.0292657, 0.311731, ],
[0.315256, -0.0493308, 0.30867, ],
[0.30997, -0.0693959, 0.305608, ],
[0.304684, -0.089461, 0.302546, ],
[0.299398, -0.109526, 0.299485, ],
[0.294111, -0.129591, 0.296423, ],
[0.288825, -0.149656, 0.293361, ],
[0.283539, -0.169722, 0.2903, ],
[0.34109, 0.0258857, 0.325328, ],
[0.335804, 0.00582053, 0.322266, ],
[0.330518, -0.0142446, 0.319205, ],
[0.325232, -0.0343097, 0.316143, ],
[0.319946, -0.0543748, 0.313082, ],
[0.314659, -0.07444, 0.31002, ],
[0.309373, -0.0945051, 0.306958, ],
[0.304087, -0.11457, 0.303897, ],
[0.298801, -0.134635, 0.300835, ],
[0.293515, -0.1547, 0.297774, ],
[0.351066, 0.0409067, 0.332802, ],
[0.34578, 0.0208416, 0.32974, ],
[0.340494, 0.00077649, 0.326679, ],
[0.335208, -0.0192886, 0.323617, ],
[0.329921, -0.0393538, 0.320555, ],
[0.324635, -0.0594189, 0.317494, ],
[0.319349, -0.079484, 0.314432, ],
[0.314063, -0.0995491, 0.31137, ],
[0.308777, -0.119614, 0.308309, ],
[0.303491, -0.139679, 0.305247, ],
[0.361042, 0.0559278, 0.340275, ],
[0.355756, 0.0358627, 0.337214, ],
[0.350469, 0.0157976, 0.334152, ],
[0.345183, -0.00426755, 0.331091, ],
[0.339897, -0.0243327, 0.328029, ],
[0.334611, -0.0443978, 0.324967, ],
[0.329325, -0.0644629, 0.321906, ],
[0.324039, -0.084528, 0.318844, ],
[0.318753, -0.104593, 0.315783, ],
[0.313466, -0.124658, 0.312721, ],
[0.261512, -0.0851143, 0.287615, ],
[0.256226, -0.105179, 0.284553, ],
[0.25094, -0.125245, 0.281491, ],
[0.245653, -0.14531, 0.27843, ],
[0.240367, -0.165375, 0.275368, ],
[0.235081, -0.18544, 0.272306, ],
[0.229795, -0.205505, 0.269245, ],
[0.224509, -0.22557, 0.266183, ],
[0.219223, -0.245635, 0.263122, ],
[0.213937, -0.2657, 0.26006, ],
[0.271488, -0.0700932, 0.295088, ],
[0.266201, -0.0901583, 0.292027, ],
[0.260915, -0.110223, 0.288965, ],
[0.255629, -0.130289, 0.285903, ],
[0.250343, -0.150354, 0.282842, ],
[0.245057, -0.170419, 0.27978, ],
[0.239771, -0.190484, 0.276719, ],
[0.234485, -0.210549, 0.273657, ],
[0.229199, -0.230614, 0.270595, ],
[0.223912, -0.250679, 0.267534, ],
[0.281463, -0.0550721, 0.302562, ],
[0.276177, -0.0751373, 0.2995, ],
[0.270891, -0.0952024, 0.296439, ],
[0.265605, -0.115268, 0.293377, ],
[0.260319, -0.135333, 0.290315, ],
[0.255033, -0.155398, 0.287254, ],
[0.249747, -0.175463, 0.284192, ],
[0.24446, -0.195528, 0.281131, ],
[0.239174, -0.215593, 0.278069, ],
[0.233888, -0.235658, 0.275007, ],
[0.291439, -0.0400511, 0.310036, ],
[0.286153, -0.0601162, 0.306974, ],
[0.280867, -0.0801813, 0.303912, ],
[0.275581, -0.100246, 0.300851, ],
[0.270295, -0.120312, 0.297789, ],
[0.265008, -0.140377, 0.294728, ],
[0.259722, -0.160442, 0.291666, ],
[0.254436, -0.180507, 0.288604, ],
[0.24915, -0.200572, 0.285543, ],
[0.243864, -0.220637, 0.282481, ],
[0.301415, -0.02503, 0.317509, ],
[0.296129, -0.0450951, 0.314448, ],
[0.290843, -0.0651602, 0.311386, ],
[0.285556, -0.0852253, 0.308325, ],
[0.28027, -0.10529, 0.305263, ],
[0.274984, -0.125356, 0.302201, ],
[0.269698, -0.145421, 0.29914, ],
[0.264412, -0.165486, 0.296078, ],
[0.259126, -0.185551, 0.293016, ],
[0.25384, -0.205616, 0.289955, ],
[0.311391, -0.0100089, 0.324983, ],
[0.306104, -0.030074, 0.321921, ],
[0.300818, -0.0501391, 0.31886, ],
[0.295532, -0.0702043, 0.315798, ],
[0.290246, -0.0902694, 0.312737, ],
[0.28496, -0.110335, 0.309675, ],
[0.279674, -0.1304, 0.306613, ],
[0.274388, -0.150465, 0.303552, ],
[0.269102, -0.17053, 0.30049, ],
[0.263815, -0.190595, 0.297429, ],
[0.321366, 0.00501219, 0.332457, ],
[0.31608, -0.0150529, 0.329395, ],
[0.310794, -0.0351181, 0.326334, ],
[0.305508, -0.0551832, 0.323272, ],
[0.300222, -0.0752483, 0.32021, ],
[0.294936, -0.0953134, 0.317149, ],
[0.28965, -0.115379, 0.314087, ],
[0.284363, -0.135444, 0.311025, ],
[0.279077, -0.155509, 0.307964, ],
[0.273791, -0.175574, 0.304902, ],
[0.331342, 0.0200333, 0.33993, ],
[0.326056, -3.1851e-05, 0.336869, ],
[0.32077, -0.020097, 0.333807, ],
[0.315484, -0.0401621, 0.330746, ],
[0.310198, -0.0602272, 0.327684, ],
[0.304911, -0.0802923, 0.324622, ],
[0.299625, -0.100357, 0.321561, ],
[0.294339, -0.120423, 0.318499, ],
[0.289053, -0.140488, 0.315438, ],
[0.283767, -0.160553, 0.312376, ],
[0.341318, 0.0350544, 0.347404, ],
[0.336032, 0.0149892, 0.344343, ],
[0.330746, -0.00507589, 0.341281, ],
[0.325459, -0.025141, 0.338219, ],
[0.320173, -0.0452061, 0.335158, ],
[0.314887, -0.0652713, 0.332096, ],
[0.309601, -0.0853364, 0.329035, ],
[0.304315, -0.105402, 0.325973, ],
[0.299029, -0.125467, 0.322911, ],
[0.293743, -0.145532, 0.31985, ],
[0.351294, 0.0500754, 0.354878, ],
[0.346008, 0.0300103, 0.351816, ],
[0.340721, 0.00994519, 0.348755, ],
[0.335435, -0.0101199, 0.345693, ],
[0.330149, -0.0301851, 0.342631, ],
[0.324863, -0.0502502, 0.33957, ],
[0.319577, -0.0703153, 0.336508, ],
[0.314291, -0.0903804, 0.333447, ],
[0.309005, -0.110446, 0.330385, ],
[0.303718, -0.130511, 0.327323, ],
[0.251764, -0.0909667, 0.302217, ],
[0.246478, -0.111032, 0.299155, ],
[0.241192, -0.131097, 0.296094, ],
[0.235905, -0.151162, 0.293032, ],
[0.230619, -0.171227, 0.28997, ],
[0.225333, -0.191292, 0.286909, ],
[0.220047, -0.211357, 0.283847, ],
[0.214761, -0.231423, 0.280786, ],
[0.209475, -0.251488, 0.277724, ],
[0.204189, -0.271553, 0.274662, ],
[0.26174, -0.0759456, 0.309691, ],
[0.256453, -0.0960107, 0.306629, ],
[0.251167, -0.116076, 0.303567, ],
[0.245881, -0.136141, 0.300506, ],
[0.240595, -0.156206, 0.297444, ],
[0.235309, -0.176271, 0.294383, ],
[0.230023, -0.196336, 0.291321, ],
[0.224737, -0.216401, 0.288259, ],
[0.219451, -0.236467, 0.285198, ],
[0.214164, -0.256532, 0.282136, ],
[0.271715, -0.0609245, 0.317164, ],
[0.266429, -0.0809896, 0.314103, ],
[0.261143, -0.101055, 0.311041, ],
[0.255857, -0.12112, 0.30798, ],
[0.250571, -0.141185, 0.304918, ],
[0.245285, -0.16125, 0.301856, ],
[0.239999, -0.181315, 0.298795, ],
[0.234712, -0.20138, 0.295733, ],
[0.229426, -0.221446, 0.292671, ],
[0.22414, -0.241511, 0.28961, ],
[0.281691, -0.0459034, 0.324638, ],
[0.276405, -0.0659686, 0.321576, ],
[0.271119, -0.0860337, 0.318515, ],
[0.265833, -0.106099, 0.315453, ],
[0.260547, -0.126164, 0.312392, ],
[0.25526, -0.146229, 0.30933, ],
[0.249974, -0.166294, 0.306268, ],
[0.244688, -0.186359, 0.303207, ],
[0.239402, -0.206424, 0.300145, ],
[0.234116, -0.22649, 0.297084, ],
[0.291667, -0.0308824, 0.332112, ],
[0.286381, -0.0509475, 0.32905, ],
[0.281095, -0.0710126, 0.325989, ],
[0.275808, -0.0910777, 0.322927, ],
[0.270522, -0.111143, 0.319865, ],
[0.265236, -0.131208, 0.316804, ],
[0.25995, -0.151273, 0.313742, ],
[0.254664, -0.171338, 0.31068, ],
[0.249378, -0.191403, 0.307619, ],
[0.244092, -0.211468, 0.304557, ],
[0.301643, -0.0158613, 0.339585, ],
[0.296356, -0.0359264, 0.336524, ],
[0.29107, -0.0559915, 0.333462, ],
[0.285784, -0.0760566, 0.330401, ],
[0.280498, -0.0961218, 0.327339, ],
[0.275212, -0.116187, 0.324277, ],
[0.269926, -0.136252, 0.321216, ],
[0.26464, -0.156317, 0.318154, ],
[0.259354, -0.176382, 0.315093, ],
[0.254067, -0.196447, 0.312031, ],
[0.311618, -0.000840192, 0.347059, ],
[0.306332, -0.0209053, 0.343998, ],
[0.301046, -0.0409704, 0.340936, ],
[0.29576, -0.0610356, 0.337874, ],
[0.290474, -0.0811007, 0.334813, ],
[0.285188, -0.101166, 0.331751, ],
[0.279902, -0.121231, 0.32869, ],
[0.274615, -0.141296, 0.325628, ],
[0.269329, -0.161361, 0.322566, ],
[0.264043, -0.181426, 0.319505, ],
[0.321594, 0.0141809, 0.354533, ],
[0.316308, -0.00588423, 0.351471, ],
[0.311022, -0.0259494, 0.34841, ],
[0.305736, -0.0460145, 0.345348, ],
[0.30045, -0.0660796, 0.342286, ],
[0.295163, -0.0861447, 0.339225, ],
[0.289877, -0.10621, 0.336163, ],
[0.284591, -0.126275, 0.333102, ],
[0.279305, -0.14634, 0.33004, ],
[0.274019, -0.166405, 0.326978, ],
[0.33157, 0.029202, 0.362007, ],
[0.326284, 0.00913685, 0.358945, ],
[0.320998, -0.0109283, 0.355883, ],
[0.315711, -0.0309934, 0.352822, ],
[0.310425, -0.0510585, 0.34976, ],
[0.305139, -0.0711237, 0.346699, ],
[0.299853, -0.0911888, 0.343637, ],
[0.294567, -0.111254, 0.340575, ],
[0.289281, -0.131319, 0.337514, ],
[0.283995, -0.151384, 0.334452, ],
[0.341546, 0.0442231, 0.36948, ],
[0.336259, 0.0241579, 0.366419, ],
[0.330973, 0.00409281, 0.363357, ],
[0.325687, -0.0159723, 0.360295, ],
[0.320401, -0.0360374, 0.357234, ],
[0.315115, -0.0561026, 0.354172, ],
[0.309829, -0.0761677, 0.351111, ],
[0.304543, -0.0962328, 0.348049, ],
[0.299257, -0.116298, 0.344987, ],
[0.29397, -0.136363, 0.341926, ],
[0.242016, -0.0968191, 0.316819, ],
[0.23673, -0.116884, 0.313758, ],
[0.231444, -0.136949, 0.310696, ],
[0.226157, -0.157014, 0.307635, ],
[0.220871, -0.17708, 0.304573, ],
[0.215585, -0.197145, 0.301511, ],
[0.210299, -0.21721, 0.29845, ],
[0.205013, -0.237275, 0.295388, ],
[0.199727, -0.25734, 0.292326, ],
[0.194441, -0.277405, 0.289265, ],
[0.251992, -0.081798, 0.324293, ],
[0.246705, -0.101863, 0.321231, ],
[0.241419, -0.121928, 0.31817, ],
[0.236133, -0.141993, 0.315108, ],
[0.230847, -0.162058, 0.312047, ],
[0.225561, -0.182124, 0.308985, ],
[0.220275, -0.202189, 0.305923, ],
[0.214989, -0.222254, 0.302862, ],
[0.209702, -0.242319, 0.2998, ],
[0.204416, -0.262384, 0.296739, ],
[0.261967, -0.0667769, 0.331767, ],
[0.256681, -0.086842, 0.328705, ],
[0.251395, -0.106907, 0.325644, ],
[0.246109, -0.126972, 0.322582, ],
[0.240823, -0.147037, 0.31952, ],
[0.235537, -0.167103, 0.316459, ],
[0.230251, -0.187168, 0.313397, ],
[0.224964, -0.207233, 0.310335, ],
[0.219678, -0.227298, 0.307274, ],
[0.214392, -0.247363, 0.304212, ],
[0.271943, -0.0517558, 0.33924, ],
[0.266657, -0.0718209, 0.336179, ],
[0.261371, -0.0918861, 0.333117, ],
[0.256085, -0.111951, 0.330056, ],
[0.250799, -0.132016, 0.326994, ],
[0.245512, -0.152081, 0.323932, ],
[0.240226, -0.172147, 0.320871, ],
[0.23494, -0.192212, 0.317809, ],
[0.229654, -0.212277, 0.314748, ],
[0.224368, -0.232342, 0.311686, ],
[0.281919, -0.0367347, 0.346714, ],
[0.276633, -0.0567999, 0.343653, ],
[0.271347, -0.076865, 0.340591, ],
[0.26606, -0.0969301, 0.337529, ],
[0.260774, -0.116995, 0.334468, ],
[0.255488, -0.13706, 0.331406, ],
[0.250202, -0.157125, 0.328345, ],
[0.244916, -0.177191, 0.325283, ],
[0.23963, -0.197256, 0.322221, ],
[0.234344, -0.217321, 0.31916, ],
[0.291895, -0.0217137, 0.354188, ],
[0.286608, -0.0417788, 0.351126, ],
[0.281322, -0.0618439, 0.348065, ],
[0.276036, -0.081909, 0.345003, ],
[0.27075, -0.101974, 0.341941, ],
[0.265464, -0.122039, 0.33888, ],
[0.260178, -0.142104, 0.335818, ],
[0.254892, -0.16217, 0.332757, ],
[0.249606, -0.182235, 0.329695, ],
[0.244319, -0.2023, 0.326633, ],
[0.30187, -0.00669258, 0.361662, ],
[0.296584, -0.0267577, 0.3586, ],
[0.291298, -0.0468228, 0.355538, ],
[0.286012, -0.0668879, 0.352477, ],
[0.280726, -0.0869531, 0.349415, ],
[0.27544, -0.107018, 0.346354, ],
[0.270154, -0.127083, 0.343292, ],
[0.264867, -0.147148, 0.34023, ],
[0.259581, -0.167214, 0.337169, ],
[0.254295, -0.187279, 0.334107, ],
[0.311846, 0.00832851, 0.369135, ],
[0.30656, -0.0117366, 0.366074, ],
[0.301274, -0.0318017, 0.363012, ],
[0.295988, -0.0518669, 0.35995, ],
[0.290702, -0.071932, 0.356889, ],
[0.285415, -0.0919971, 0.353827, ],
[0.280129, -0.112062, 0.350766, ],
[0.274843, -0.132127, 0.347704, ],
[0.269557, -0.152192, 0.344642, ],
[0.264271, -0.172258, 0.341581, ],
[0.321822, 0.0233496, 0.376609, ],
[0.316536, 0.00328446, 0.373547, ],
[0.31125, -0.0167807, 0.370486, ],
[0.305963, -0.0368458, 0.367424, ],
[0.300677, -0.0569109, 0.364363, ],
[0.295391, -0.076976, 0.361301, ],
[0.290105, -0.0970412, 0.358239, ],
[0.284819, -0.117106, 0.355178, ],
[0.279533, -0.137171, 0.352116, ],
[0.274247, -0.157237, 0.349055, ],
[0.331798, 0.0383707, 0.384083, ],
[0.326511, 0.0183055, 0.381021, ],
[0.321225, -0.00175958, 0.37796, ],
[0.315939, -0.0218247, 0.374898, ],
[0.310653, -0.0418898, 0.371836, ],
[0.305367, -0.061955, 0.368775, ],
[0.300081, -0.0820201, 0.365713, ],
[0.294795, -0.102085, 0.362651, ],
[0.289509, -0.12215, 0.35959, ],
[0.284222, -0.142215, 0.356528, ],
[0.232268, -0.102671, 0.331422, ],
[0.226982, -0.122737, 0.32836, ],
[0.221696, -0.142802, 0.325299, ],
[0.216409, -0.162867, 0.322237, ],
[0.211123, -0.182932, 0.319175, ],
[0.205837, -0.202997, 0.316114, ],
[0.200551, -0.223062, 0.313052, ],
[0.195265, -0.243127, 0.30999, ],
[0.189979, -0.263192, 0.306929, ],
[0.184693, -0.283258, 0.303867, ],
[0.242244, -0.0876504, 0.338895, ],
[0.236957, -0.107715, 0.335834, ],
[0.231671, -0.127781, 0.332772, ],
[0.226385, -0.147846, 0.329711, ],
[0.221099, -0.167911, 0.326649, ],
[0.215813, -0.187976, 0.323587, ],
[0.210527, -0.208041, 0.320526, ],
[0.205241, -0.228106, 0.317464, ],
[0.199954, -0.248171, 0.314403, ],
[0.194668, -0.268236, 0.311341, ],
[0.252219, -0.0726293, 0.346369, ],
[0.246933, -0.0926944, 0.343308, ],
[0.241647, -0.11276, 0.340246, ],
[0.236361, -0.132825, 0.337184, ],
[0.231075, -0.15289, 0.334123, ],
[0.225789, -0.172955, 0.331061, ],
[0.220502, -0.19302, 0.328, ],
[0.215216, -0.213085, 0.324938, ],
[0.20993, -0.23315, 0.321876, ],
[0.204644, -0.253215, 0.318815, ],
[0.262195, -0.0576082, 0.353843, ],
[0.256909, -0.0776733, 0.350781, ],
[0.251623, -0.0977385, 0.34772, ],
[0.246337, -0.117804, 0.344658, ],
[0.241051, -0.137869, 0.341596, ],
[0.235764, -0.157934, 0.338535, ],
[0.230478, -0.177999, 0.335473, ],
[0.225192, -0.198064, 0.332412, ],
[0.219906, -0.218129, 0.32935, ],
[0.21462, -0.238194, 0.326288, ],
[0.272171, -0.0425871, 0.361317, ],
[0.266885, -0.0626522, 0.358255, ],
[0.261599, -0.0827174, 0.355193, ],
[0.256312, -0.102782, 0.352132, ],
[0.251026, -0.122848, 0.34907, ],
[0.24574, -0.142913, 0.346009, ],
[0.240454, -0.162978, 0.342947, ],
[0.235168, -0.183043, 0.339885, ],
[0.229882, -0.203108, 0.336824, ],
[0.224596, -0.223173, 0.333762, ],
[0.282147, -0.027566, 0.36879, ],
[0.27686, -0.0476312, 0.365729, ],
[0.271574, -0.0676963, 0.362667, ],
[0.266288, -0.0877614, 0.359605, ],
[0.261002, -0.107827, 0.356544, ],
[0.255716, -0.127892, 0.353482, ],
[0.25043, -0.147957, 0.350421, ],
[0.245144, -0.168022, 0.347359, ],
[0.239857, -0.188087, 0.344297, ],
[0.234571, -0.208152, 0.341236, ],
[0.292122, -0.012545, 0.376264, ],
[0.286836, -0.0326101, 0.373202, ],
[0.28155, -0.0526752, 0.370141, ],
[0.276264, -0.0727403, 0.367079, ],
[0.270978, -0.0928055, 0.364018, ],
[0.265692, -0.112871, 0.360956, ],
[0.260406, -0.132936, 0.357894, ],
[0.255119, -0.153001, 0.354833, ],
[0.249833, -0.173066, 0.351771, ],
[0.244547, -0.193131, 0.34871, ],
[0.302098, 0.00247612, 0.383738, ],
[0.296812, -0.017589, 0.380676, ],
[0.291526, -0.0376541, 0.377615, ],
[0.28624, -0.0577193, 0.374553, ],
[0.280954, -0.0777844, 0.371491, ],
[0.275667, -0.0978495, 0.36843, ],
[0.270381, -0.117915, 0.365368, ],
[0.265095, -0.13798, 0.362306, ],
[0.259809, -0.158045, 0.359245, ],
[0.254523, -0.17811, 0.356183, ],
[0.312074, 0.0174972, 0.391211, ],
[0.306788, -0.00256792, 0.38815, ],
[0.301502, -0.022633, 0.385088, ],
[0.296215, -0.0426982, 0.382027, ],
[0.290929, -0.0627633, 0.378965, ],
[0.285643, -0.0828284, 0.375903, ],
[0.280357, -0.102894, 0.372842, ],
[0.275071, -0.122959, 0.36978, ],
[0.269785, -0.143024, 0.366719, ],
[0.264499, -0.163089, 0.363657, ],
[0.32205, 0.0325183, 0.398685, ],
[0.316763, 0.0124532, 0.395624, ],
[0.311477, -0.00761196, 0.392562, ],
[0.306191, -0.0276771, 0.3895, ],
[0.300905, -0.0477422, 0.386439, ],
[0.295619, -0.0678073, 0.383377, ],
[0.290333, -0.0878725, 0.380315, ],
[0.285047, -0.107938, 0.377254, ],
[0.279761, -0.128003, 0.374192, ],
[0.274474, -0.148068, 0.371131, ],
])


In [62]:
micro_us = np.array([[0.32, -0.05, 0.2, ],
[0.319444, -0.0522222, 0.198889, ],
[0.318889, -0.0544444, 0.197778, ],
[0.318333, -0.0566667, 0.196667, ],
[0.317778, -0.0588889, 0.195556, ],
[0.317222, -0.0611111, 0.194444, ],
[0.316667, -0.0633333, 0.193333, ],
[0.316111, -0.0655556, 0.192222, ],
[0.315556, -0.0677778, 0.191111, ],
[0.315, -0.07, 0.19, ],
[0.331111, -0.0477778, 0.199444, ],
[0.330556, -0.05, 0.198333, ],
[0.33, -0.0522222, 0.197222, ],
[0.329444, -0.0544444, 0.196111, ],
[0.328889, -0.0566667, 0.195, ],
[0.328333, -0.0588889, 0.193889, ],
[0.327778, -0.0611111, 0.192778, ],
[0.327222, -0.0633333, 0.191667, ],
[0.326667, -0.0655556, 0.190556, ],
[0.326111, -0.0677778, 0.189444, ],
[0.342222, -0.0455556, 0.198889, ],
[0.341667, -0.0477778, 0.197778, ],
[0.341111, -0.05, 0.196667, ],
[0.340556, -0.0522222, 0.195556, ],
[0.34, -0.0544444, 0.194444, ],
[0.339444, -0.0566667, 0.193333, ],
[0.338889, -0.0588889, 0.192222, ],
[0.338333, -0.0611111, 0.191111, ],
[0.337778, -0.0633333, 0.19, ],
[0.337222, -0.0655556, 0.188889, ],
[0.353333, -0.0433333, 0.198333, ],
[0.352778, -0.0455556, 0.197222, ],
[0.352222, -0.0477778, 0.196111, ],
[0.351667, -0.05, 0.195, ],
[0.351111, -0.0522222, 0.193889, ],
[0.350556, -0.0544444, 0.192778, ],
[0.35, -0.0566667, 0.191667, ],
[0.349444, -0.0588889, 0.190556, ],
[0.348889, -0.0611111, 0.189444, ],
[0.348333, -0.0633333, 0.188333, ],
[0.364444, -0.0411111, 0.197778, ],
[0.363889, -0.0433333, 0.196667, ],
[0.363333, -0.0455556, 0.195556, ],
[0.362778, -0.0477778, 0.194444, ],
[0.362222, -0.05, 0.193333, ],
[0.361667, -0.0522222, 0.192222, ],
[0.361111, -0.0544444, 0.191111, ],
[0.360556, -0.0566667, 0.19, ],
[0.36, -0.0588889, 0.188889, ],
[0.359444, -0.0611111, 0.187778, ],
[0.375556, -0.0388889, 0.197222, ],
[0.375, -0.0411111, 0.196111, ],
[0.374444, -0.0433333, 0.195, ],
[0.373889, -0.0455556, 0.193889, ],
[0.373333, -0.0477778, 0.192778, ],
[0.372778, -0.05, 0.191667, ],
[0.372222, -0.0522222, 0.190556, ],
[0.371667, -0.0544444, 0.189444, ],
[0.371111, -0.0566667, 0.188333, ],
[0.370556, -0.0588889, 0.187222, ],
[0.386667, -0.0366667, 0.196667, ],
[0.386111, -0.0388889, 0.195556, ],
[0.385556, -0.0411111, 0.194444, ],
[0.385, -0.0433333, 0.193333, ],
[0.384444, -0.0455556, 0.192222, ],
[0.383889, -0.0477778, 0.191111, ],
[0.383333, -0.05, 0.19, ],
[0.382778, -0.0522222, 0.188889, ],
[0.382222, -0.0544444, 0.187778, ],
[0.381667, -0.0566667, 0.186667, ],
[0.397778, -0.0344444, 0.196111, ],
[0.397222, -0.0366667, 0.195, ],
[0.396667, -0.0388889, 0.193889, ],
[0.396111, -0.0411111, 0.192778, ],
[0.395556, -0.0433333, 0.191667, ],
[0.395, -0.0455556, 0.190556, ],
[0.394444, -0.0477778, 0.189444, ],
[0.393889, -0.05, 0.188333, ],
[0.393333, -0.0522222, 0.187222, ],
[0.392778, -0.0544444, 0.186111, ],
[0.408889, -0.0322222, 0.195556, ],
[0.408333, -0.0344444, 0.194444, ],
[0.407778, -0.0366667, 0.193333, ],
[0.407222, -0.0388889, 0.192222, ],
[0.406667, -0.0411111, 0.191111, ],
[0.406111, -0.0433333, 0.19, ],
[0.405556, -0.0455556, 0.188889, ],
[0.405, -0.0477778, 0.187778, ],
[0.404444, -0.05, 0.186667, ],
[0.403889, -0.0522222, 0.185556, ],
[0.42, -0.03, 0.195, ],
[0.419444, -0.0322222, 0.193889, ],
[0.418889, -0.0344444, 0.192778, ],
[0.418333, -0.0366667, 0.191667, ],
[0.417778, -0.0388889, 0.190556, ],
[0.417222, -0.0411111, 0.189444, ],
[0.416667, -0.0433333, 0.188333, ],
[0.416111, -0.0455556, 0.187222, ],
[0.415556, -0.0477778, 0.186111, ],
[0.415, -0.05, 0.185, ],
[0.322222, -0.0544444, 0.197778, ],
[0.321667, -0.0566667, 0.196667, ],
[0.321111, -0.0588889, 0.195556, ],
[0.320556, -0.0611111, 0.194444, ],
[0.32, -0.0633333, 0.193333, ],
[0.319444, -0.0655556, 0.192222, ],
[0.318889, -0.0677778, 0.191111, ],
[0.318333, -0.07, 0.19, ],
[0.317778, -0.0722222, 0.188889, ],
[0.317222, -0.0744444, 0.187778, ],
[0.333333, -0.0522222, 0.197222, ],
[0.332778, -0.0544444, 0.196111, ],
[0.332222, -0.0566667, 0.195, ],
[0.331667, -0.0588889, 0.193889, ],
[0.331111, -0.0611111, 0.192778, ],
[0.330556, -0.0633333, 0.191667, ],
[0.33, -0.0655556, 0.190556, ],
[0.329444, -0.0677778, 0.189444, ],
[0.328889, -0.07, 0.188333, ],
[0.328333, -0.0722222, 0.187222, ],
[0.344444, -0.05, 0.196667, ],
[0.343889, -0.0522222, 0.195556, ],
[0.343333, -0.0544444, 0.194444, ],
[0.342778, -0.0566667, 0.193333, ],
[0.342222, -0.0588889, 0.192222, ],
[0.341667, -0.0611111, 0.191111, ],
[0.341111, -0.0633333, 0.19, ],
[0.340556, -0.0655556, 0.188889, ],
[0.34, -0.0677778, 0.187778, ],
[0.339444, -0.07, 0.186667, ],
[0.355556, -0.0477778, 0.196111, ],
[0.355, -0.05, 0.195, ],
[0.354444, -0.0522222, 0.193889, ],
[0.353889, -0.0544444, 0.192778, ],
[0.353333, -0.0566667, 0.191667, ],
[0.352778, -0.0588889, 0.190556, ],
[0.352222, -0.0611111, 0.189444, ],
[0.351667, -0.0633333, 0.188333, ],
[0.351111, -0.0655556, 0.187222, ],
[0.350556, -0.0677778, 0.186111, ],
[0.366667, -0.0455556, 0.195556, ],
[0.366111, -0.0477778, 0.194444, ],
[0.365556, -0.05, 0.193333, ],
[0.365, -0.0522222, 0.192222, ],
[0.364444, -0.0544444, 0.191111, ],
[0.363889, -0.0566667, 0.19, ],
[0.363333, -0.0588889, 0.188889, ],
[0.362778, -0.0611111, 0.187778, ],
[0.362222, -0.0633333, 0.186667, ],
[0.361667, -0.0655556, 0.185556, ],
[0.377778, -0.0433333, 0.195, ],
[0.377222, -0.0455556, 0.193889, ],
[0.376667, -0.0477778, 0.192778, ],
[0.376111, -0.05, 0.191667, ],
[0.375556, -0.0522222, 0.190556, ],
[0.375, -0.0544444, 0.189444, ],
[0.374444, -0.0566667, 0.188333, ],
[0.373889, -0.0588889, 0.187222, ],
[0.373333, -0.0611111, 0.186111, ],
[0.372778, -0.0633333, 0.185, ],
[0.388889, -0.0411111, 0.194444, ],
[0.388333, -0.0433333, 0.193333, ],
[0.387778, -0.0455556, 0.192222, ],
[0.387222, -0.0477778, 0.191111, ],
[0.386667, -0.05, 0.19, ],
[0.386111, -0.0522222, 0.188889, ],
[0.385556, -0.0544444, 0.187778, ],
[0.385, -0.0566667, 0.186667, ],
[0.384444, -0.0588889, 0.185556, ],
[0.383889, -0.0611111, 0.184444, ],
[0.4, -0.0388889, 0.193889, ],
[0.399444, -0.0411111, 0.192778, ],
[0.398889, -0.0433333, 0.191667, ],
[0.398333, -0.0455556, 0.190556, ],
[0.397778, -0.0477778, 0.189444, ],
[0.397222, -0.05, 0.188333, ],
[0.396667, -0.0522222, 0.187222, ],
[0.396111, -0.0544444, 0.186111, ],
[0.395556, -0.0566667, 0.185, ],
[0.395, -0.0588889, 0.183889, ],
[0.411111, -0.0366667, 0.193333, ],
[0.410556, -0.0388889, 0.192222, ],
[0.41, -0.0411111, 0.191111, ],
[0.409444, -0.0433333, 0.19, ],
[0.408889, -0.0455556, 0.188889, ],
[0.408333, -0.0477778, 0.187778, ],
[0.407778, -0.05, 0.186667, ],
[0.407222, -0.0522222, 0.185556, ],
[0.406667, -0.0544444, 0.184444, ],
[0.406111, -0.0566667, 0.183333, ],
[0.422222, -0.0344444, 0.192778, ],
[0.421667, -0.0366667, 0.191667, ],
[0.421111, -0.0388889, 0.190556, ],
[0.420556, -0.0411111, 0.189444, ],
[0.42, -0.0433333, 0.188333, ],
[0.419444, -0.0455556, 0.187222, ],
[0.418889, -0.0477778, 0.186111, ],
[0.418333, -0.05, 0.185, ],
[0.417778, -0.0522222, 0.183889, ],
[0.417222, -0.0544444, 0.182778, ],
[0.324444, -0.0588889, 0.195556, ],
[0.323889, -0.0611111, 0.194444, ],
[0.323333, -0.0633333, 0.193333, ],
[0.322778, -0.0655556, 0.192222, ],
[0.322222, -0.0677778, 0.191111, ],
[0.321667, -0.07, 0.19, ],
[0.321111, -0.0722222, 0.188889, ],
[0.320556, -0.0744444, 0.187778, ],
[0.32, -0.0766667, 0.186667, ],
[0.319444, -0.0788889, 0.185556, ],
[0.335556, -0.0566667, 0.195, ],
[0.335, -0.0588889, 0.193889, ],
[0.334444, -0.0611111, 0.192778, ],
[0.333889, -0.0633333, 0.191667, ],
[0.333333, -0.0655556, 0.190556, ],
[0.332778, -0.0677778, 0.189444, ],
[0.332222, -0.07, 0.188333, ],
[0.331667, -0.0722222, 0.187222, ],
[0.331111, -0.0744444, 0.186111, ],
[0.330556, -0.0766667, 0.185, ],
[0.346667, -0.0544444, 0.194444, ],
[0.346111, -0.0566667, 0.193333, ],
[0.345556, -0.0588889, 0.192222, ],
[0.345, -0.0611111, 0.191111, ],
[0.344444, -0.0633333, 0.19, ],
[0.343889, -0.0655556, 0.188889, ],
[0.343333, -0.0677778, 0.187778, ],
[0.342778, -0.07, 0.186667, ],
[0.342222, -0.0722222, 0.185556, ],
[0.341667, -0.0744444, 0.184444, ],
[0.357778, -0.0522222, 0.193889, ],
[0.357222, -0.0544444, 0.192778, ],
[0.356667, -0.0566667, 0.191667, ],
[0.356111, -0.0588889, 0.190556, ],
[0.355556, -0.0611111, 0.189444, ],
[0.355, -0.0633333, 0.188333, ],
[0.354444, -0.0655556, 0.187222, ],
[0.353889, -0.0677778, 0.186111, ],
[0.353333, -0.07, 0.185, ],
[0.352778, -0.0722222, 0.183889, ],
[0.368889, -0.05, 0.193333, ],
[0.368333, -0.0522222, 0.192222, ],
[0.367778, -0.0544444, 0.191111, ],
[0.367222, -0.0566667, 0.19, ],
[0.366667, -0.0588889, 0.188889, ],
[0.366111, -0.0611111, 0.187778, ],
[0.365556, -0.0633333, 0.186667, ],
[0.365, -0.0655556, 0.185556, ],
[0.364444, -0.0677778, 0.184444, ],
[0.363889, -0.07, 0.183333, ],
[0.38, -0.0477778, 0.192778, ],
[0.379444, -0.05, 0.191667, ],
[0.378889, -0.0522222, 0.190556, ],
[0.378333, -0.0544444, 0.189444, ],
[0.377778, -0.0566667, 0.188333, ],
[0.377222, -0.0588889, 0.187222, ],
[0.376667, -0.0611111, 0.186111, ],
[0.376111, -0.0633333, 0.185, ],
[0.375556, -0.0655556, 0.183889, ],
[0.375, -0.0677778, 0.182778, ],
[0.391111, -0.0455556, 0.192222, ],
[0.390556, -0.0477778, 0.191111, ],
[0.39, -0.05, 0.19, ],
[0.389444, -0.0522222, 0.188889, ],
[0.388889, -0.0544444, 0.187778, ],
[0.388333, -0.0566667, 0.186667, ],
[0.387778, -0.0588889, 0.185556, ],
[0.387222, -0.0611111, 0.184444, ],
[0.386667, -0.0633333, 0.183333, ],
[0.386111, -0.0655556, 0.182222, ],
[0.402222, -0.0433333, 0.191667, ],
[0.401667, -0.0455556, 0.190556, ],
[0.401111, -0.0477778, 0.189444, ],
[0.400556, -0.05, 0.188333, ],
[0.4, -0.0522222, 0.187222, ],
[0.399444, -0.0544444, 0.186111, ],
[0.398889, -0.0566667, 0.185, ],
[0.398333, -0.0588889, 0.183889, ],
[0.397778, -0.0611111, 0.182778, ],
[0.397222, -0.0633333, 0.181667, ],
[0.413333, -0.0411111, 0.191111, ],
[0.412778, -0.0433333, 0.19, ],
[0.412222, -0.0455556, 0.188889, ],
[0.411667, -0.0477778, 0.187778, ],
[0.411111, -0.05, 0.186667, ],
[0.410556, -0.0522222, 0.185556, ],
[0.41, -0.0544444, 0.184444, ],
[0.409444, -0.0566667, 0.183333, ],
[0.408889, -0.0588889, 0.182222, ],
[0.408333, -0.0611111, 0.181111, ],
[0.424444, -0.0388889, 0.190556, ],
[0.423889, -0.0411111, 0.189444, ],
[0.423333, -0.0433333, 0.188333, ],
[0.422778, -0.0455556, 0.187222, ],
[0.422222, -0.0477778, 0.186111, ],
[0.421667, -0.05, 0.185, ],
[0.421111, -0.0522222, 0.183889, ],
[0.420556, -0.0544444, 0.182778, ],
[0.42, -0.0566667, 0.181667, ],
[0.419444, -0.0588889, 0.180556, ],
[0.326667, -0.0633333, 0.193333, ],
[0.326111, -0.0655556, 0.192222, ],
[0.325556, -0.0677778, 0.191111, ],
[0.325, -0.07, 0.19, ],
[0.324444, -0.0722222, 0.188889, ],
[0.323889, -0.0744444, 0.187778, ],
[0.323333, -0.0766667, 0.186667, ],
[0.322778, -0.0788889, 0.185556, ],
[0.322222, -0.0811111, 0.184444, ],
[0.321667, -0.0833333, 0.183333, ],
[0.337778, -0.0611111, 0.192778, ],
[0.337222, -0.0633333, 0.191667, ],
[0.336667, -0.0655556, 0.190556, ],
[0.336111, -0.0677778, 0.189444, ],
[0.335556, -0.07, 0.188333, ],
[0.335, -0.0722222, 0.187222, ],
[0.334444, -0.0744444, 0.186111, ],
[0.333889, -0.0766667, 0.185, ],
[0.333333, -0.0788889, 0.183889, ],
[0.332778, -0.0811111, 0.182778, ],
[0.348889, -0.0588889, 0.192222, ],
[0.348333, -0.0611111, 0.191111, ],
[0.347778, -0.0633333, 0.19, ],
[0.347222, -0.0655556, 0.188889, ],
[0.346667, -0.0677778, 0.187778, ],
[0.346111, -0.07, 0.186667, ],
[0.345556, -0.0722222, 0.185556, ],
[0.345, -0.0744444, 0.184444, ],
[0.344444, -0.0766667, 0.183333, ],
[0.343889, -0.0788889, 0.182222, ],
[0.36, -0.0566667, 0.191667, ],
[0.359444, -0.0588889, 0.190556, ],
[0.358889, -0.0611111, 0.189444, ],
[0.358333, -0.0633333, 0.188333, ],
[0.357778, -0.0655556, 0.187222, ],
[0.357222, -0.0677778, 0.186111, ],
[0.356667, -0.07, 0.185, ],
[0.356111, -0.0722222, 0.183889, ],
[0.355556, -0.0744444, 0.182778, ],
[0.355, -0.0766667, 0.181667, ],
[0.371111, -0.0544444, 0.191111, ],
[0.370556, -0.0566667, 0.19, ],
[0.37, -0.0588889, 0.188889, ],
[0.369444, -0.0611111, 0.187778, ],
[0.368889, -0.0633333, 0.186667, ],
[0.368333, -0.0655556, 0.185556, ],
[0.367778, -0.0677778, 0.184444, ],
[0.367222, -0.07, 0.183333, ],
[0.366667, -0.0722222, 0.182222, ],
[0.366111, -0.0744444, 0.181111, ],
[0.382222, -0.0522222, 0.190556, ],
[0.381667, -0.0544444, 0.189444, ],
[0.381111, -0.0566667, 0.188333, ],
[0.380556, -0.0588889, 0.187222, ],
[0.38, -0.0611111, 0.186111, ],
[0.379444, -0.0633333, 0.185, ],
[0.378889, -0.0655556, 0.183889, ],
[0.378333, -0.0677778, 0.182778, ],
[0.377778, -0.07, 0.181667, ],
[0.377222, -0.0722222, 0.180556, ],
[0.393333, -0.05, 0.19, ],
[0.392778, -0.0522222, 0.188889, ],
[0.392222, -0.0544444, 0.187778, ],
[0.391667, -0.0566667, 0.186667, ],
[0.391111, -0.0588889, 0.185556, ],
[0.390556, -0.0611111, 0.184444, ],
[0.39, -0.0633333, 0.183333, ],
[0.389444, -0.0655556, 0.182222, ],
[0.388889, -0.0677778, 0.181111, ],
[0.388333, -0.07, 0.18, ],
[0.404444, -0.0477778, 0.189444, ],
[0.403889, -0.05, 0.188333, ],
[0.403333, -0.0522222, 0.187222, ],
[0.402778, -0.0544444, 0.186111, ],
[0.402222, -0.0566667, 0.185, ],
[0.401667, -0.0588889, 0.183889, ],
[0.401111, -0.0611111, 0.182778, ],
[0.400556, -0.0633333, 0.181667, ],
[0.4, -0.0655556, 0.180556, ],
[0.399444, -0.0677778, 0.179444, ],
[0.415556, -0.0455556, 0.188889, ],
[0.415, -0.0477778, 0.187778, ],
[0.414444, -0.05, 0.186667, ],
[0.413889, -0.0522222, 0.185556, ],
[0.413333, -0.0544444, 0.184444, ],
[0.412778, -0.0566667, 0.183333, ],
[0.412222, -0.0588889, 0.182222, ],
[0.411667, -0.0611111, 0.181111, ],
[0.411111, -0.0633333, 0.18, ],
[0.410556, -0.0655556, 0.178889, ],
[0.426667, -0.0433333, 0.188333, ],
[0.426111, -0.0455556, 0.187222, ],
[0.425556, -0.0477778, 0.186111, ],
[0.425, -0.05, 0.185, ],
[0.424444, -0.0522222, 0.183889, ],
[0.423889, -0.0544444, 0.182778, ],
[0.423333, -0.0566667, 0.181667, ],
[0.422778, -0.0588889, 0.180556, ],
[0.422222, -0.0611111, 0.179444, ],
[0.421667, -0.0633333, 0.178333, ],
[0.328889, -0.0677778, 0.191111, ],
[0.328333, -0.07, 0.19, ],
[0.327778, -0.0722222, 0.188889, ],
[0.327222, -0.0744444, 0.187778, ],
[0.326667, -0.0766667, 0.186667, ],
[0.326111, -0.0788889, 0.185556, ],
[0.325556, -0.0811111, 0.184444, ],
[0.325, -0.0833333, 0.183333, ],
[0.324444, -0.0855556, 0.182222, ],
[0.323889, -0.0877778, 0.181111, ],
[0.34, -0.0655556, 0.190556, ],
[0.339444, -0.0677778, 0.189444, ],
[0.338889, -0.07, 0.188333, ],
[0.338333, -0.0722222, 0.187222, ],
[0.337778, -0.0744444, 0.186111, ],
[0.337222, -0.0766667, 0.185, ],
[0.336667, -0.0788889, 0.183889, ],
[0.336111, -0.0811111, 0.182778, ],
[0.335556, -0.0833333, 0.181667, ],
[0.335, -0.0855556, 0.180556, ],
[0.351111, -0.0633333, 0.19, ],
[0.350556, -0.0655556, 0.188889, ],
[0.35, -0.0677778, 0.187778, ],
[0.349444, -0.07, 0.186667, ],
[0.348889, -0.0722222, 0.185556, ],
[0.348333, -0.0744444, 0.184444, ],
[0.347778, -0.0766667, 0.183333, ],
[0.347222, -0.0788889, 0.182222, ],
[0.346667, -0.0811111, 0.181111, ],
[0.346111, -0.0833333, 0.18, ],
[0.362222, -0.0611111, 0.189444, ],
[0.361667, -0.0633333, 0.188333, ],
[0.361111, -0.0655556, 0.187222, ],
[0.360556, -0.0677778, 0.186111, ],
[0.36, -0.07, 0.185, ],
[0.359444, -0.0722222, 0.183889, ],
[0.358889, -0.0744444, 0.182778, ],
[0.358333, -0.0766667, 0.181667, ],
[0.357778, -0.0788889, 0.180556, ],
[0.357222, -0.0811111, 0.179444, ],
[0.373333, -0.0588889, 0.188889, ],
[0.372778, -0.0611111, 0.187778, ],
[0.372222, -0.0633333, 0.186667, ],
[0.371667, -0.0655556, 0.185556, ],
[0.371111, -0.0677778, 0.184444, ],
[0.370556, -0.07, 0.183333, ],
[0.37, -0.0722222, 0.182222, ],
[0.369444, -0.0744444, 0.181111, ],
[0.368889, -0.0766667, 0.18, ],
[0.368333, -0.0788889, 0.178889, ],
[0.384444, -0.0566667, 0.188333, ],
[0.383889, -0.0588889, 0.187222, ],
[0.383333, -0.0611111, 0.186111, ],
[0.382778, -0.0633333, 0.185, ],
[0.382222, -0.0655556, 0.183889, ],
[0.381667, -0.0677778, 0.182778, ],
[0.381111, -0.07, 0.181667, ],
[0.380556, -0.0722222, 0.180556, ],
[0.38, -0.0744444, 0.179444, ],
[0.379444, -0.0766667, 0.178333, ],
[0.395556, -0.0544444, 0.187778, ],
[0.395, -0.0566667, 0.186667, ],
[0.394444, -0.0588889, 0.185556, ],
[0.393889, -0.0611111, 0.184444, ],
[0.393333, -0.0633333, 0.183333, ],
[0.392778, -0.0655556, 0.182222, ],
[0.392222, -0.0677778, 0.181111, ],
[0.391667, -0.07, 0.18, ],
[0.391111, -0.0722222, 0.178889, ],
[0.390556, -0.0744444, 0.177778, ],
[0.406667, -0.0522222, 0.187222, ],
[0.406111, -0.0544444, 0.186111, ],
[0.405556, -0.0566667, 0.185, ],
[0.405, -0.0588889, 0.183889, ],
[0.404444, -0.0611111, 0.182778, ],
[0.403889, -0.0633333, 0.181667, ],
[0.403333, -0.0655556, 0.180556, ],
[0.402778, -0.0677778, 0.179444, ],
[0.402222, -0.07, 0.178333, ],
[0.401667, -0.0722222, 0.177222, ],
[0.417778, -0.05, 0.186667, ],
[0.417222, -0.0522222, 0.185556, ],
[0.416667, -0.0544444, 0.184444, ],
[0.416111, -0.0566667, 0.183333, ],
[0.415556, -0.0588889, 0.182222, ],
[0.415, -0.0611111, 0.181111, ],
[0.414444, -0.0633333, 0.18, ],
[0.413889, -0.0655556, 0.178889, ],
[0.413333, -0.0677778, 0.177778, ],
[0.412778, -0.07, 0.176667, ],
[0.428889, -0.0477778, 0.186111, ],
[0.428333, -0.05, 0.185, ],
[0.427778, -0.0522222, 0.183889, ],
[0.427222, -0.0544444, 0.182778, ],
[0.426667, -0.0566667, 0.181667, ],
[0.426111, -0.0588889, 0.180556, ],
[0.425556, -0.0611111, 0.179444, ],
[0.425, -0.0633333, 0.178333, ],
[0.424444, -0.0655556, 0.177222, ],
[0.423889, -0.0677778, 0.176111, ],
[0.331111, -0.0722222, 0.188889, ],
[0.330556, -0.0744444, 0.187778, ],
[0.33, -0.0766667, 0.186667, ],
[0.329444, -0.0788889, 0.185556, ],
[0.328889, -0.0811111, 0.184444, ],
[0.328333, -0.0833333, 0.183333, ],
[0.327778, -0.0855556, 0.182222, ],
[0.327222, -0.0877778, 0.181111, ],
[0.326667, -0.09, 0.18, ],
[0.326111, -0.0922222, 0.178889, ],
[0.342222, -0.07, 0.188333, ],
[0.341667, -0.0722222, 0.187222, ],
[0.341111, -0.0744444, 0.186111, ],
[0.340556, -0.0766667, 0.185, ],
[0.34, -0.0788889, 0.183889, ],
[0.339444, -0.0811111, 0.182778, ],
[0.338889, -0.0833333, 0.181667, ],
[0.338333, -0.0855556, 0.180556, ],
[0.337778, -0.0877778, 0.179444, ],
[0.337222, -0.09, 0.178333, ],
[0.353333, -0.0677778, 0.187778, ],
[0.352778, -0.07, 0.186667, ],
[0.352222, -0.0722222, 0.185556, ],
[0.351667, -0.0744444, 0.184444, ],
[0.351111, -0.0766667, 0.183333, ],
[0.350556, -0.0788889, 0.182222, ],
[0.35, -0.0811111, 0.181111, ],
[0.349444, -0.0833333, 0.18, ],
[0.348889, -0.0855556, 0.178889, ],
[0.348333, -0.0877778, 0.177778, ],
[0.364444, -0.0655556, 0.187222, ],
[0.363889, -0.0677778, 0.186111, ],
[0.363333, -0.07, 0.185, ],
[0.362778, -0.0722222, 0.183889, ],
[0.362222, -0.0744444, 0.182778, ],
[0.361667, -0.0766667, 0.181667, ],
[0.361111, -0.0788889, 0.180556, ],
[0.360556, -0.0811111, 0.179444, ],
[0.36, -0.0833333, 0.178333, ],
[0.359444, -0.0855556, 0.177222, ],
[0.375556, -0.0633333, 0.186667, ],
[0.375, -0.0655556, 0.185556, ],
[0.374444, -0.0677778, 0.184444, ],
[0.373889, -0.07, 0.183333, ],
[0.373333, -0.0722222, 0.182222, ],
[0.372778, -0.0744444, 0.181111, ],
[0.372222, -0.0766667, 0.18, ],
[0.371667, -0.0788889, 0.178889, ],
[0.371111, -0.0811111, 0.177778, ],
[0.370556, -0.0833333, 0.176667, ],
[0.386667, -0.0611111, 0.186111, ],
[0.386111, -0.0633333, 0.185, ],
[0.385556, -0.0655556, 0.183889, ],
[0.385, -0.0677778, 0.182778, ],
[0.384444, -0.07, 0.181667, ],
[0.383889, -0.0722222, 0.180556, ],
[0.383333, -0.0744444, 0.179444, ],
[0.382778, -0.0766667, 0.178333, ],
[0.382222, -0.0788889, 0.177222, ],
[0.381667, -0.0811111, 0.176111, ],
[0.397778, -0.0588889, 0.185556, ],
[0.397222, -0.0611111, 0.184444, ],
[0.396667, -0.0633333, 0.183333, ],
[0.396111, -0.0655556, 0.182222, ],
[0.395556, -0.0677778, 0.181111, ],
[0.395, -0.07, 0.18, ],
[0.394444, -0.0722222, 0.178889, ],
[0.393889, -0.0744444, 0.177778, ],
[0.393333, -0.0766667, 0.176667, ],
[0.392778, -0.0788889, 0.175556, ],
[0.408889, -0.0566667, 0.185, ],
[0.408333, -0.0588889, 0.183889, ],
[0.407778, -0.0611111, 0.182778, ],
[0.407222, -0.0633333, 0.181667, ],
[0.406667, -0.0655556, 0.180556, ],
[0.406111, -0.0677778, 0.179444, ],
[0.405556, -0.07, 0.178333, ],
[0.405, -0.0722222, 0.177222, ],
[0.404444, -0.0744444, 0.176111, ],
[0.403889, -0.0766667, 0.175, ],
[0.42, -0.0544444, 0.184444, ],
[0.419444, -0.0566667, 0.183333, ],
[0.418889, -0.0588889, 0.182222, ],
[0.418333, -0.0611111, 0.181111, ],
[0.417778, -0.0633333, 0.18, ],
[0.417222, -0.0655556, 0.178889, ],
[0.416667, -0.0677778, 0.177778, ],
[0.416111, -0.07, 0.176667, ],
[0.415556, -0.0722222, 0.175556, ],
[0.415, -0.0744444, 0.174444, ],
[0.431111, -0.0522222, 0.183889, ],
[0.430556, -0.0544444, 0.182778, ],
[0.43, -0.0566667, 0.181667, ],
[0.429444, -0.0588889, 0.180556, ],
[0.428889, -0.0611111, 0.179444, ],
[0.428333, -0.0633333, 0.178333, ],
[0.427778, -0.0655556, 0.177222, ],
[0.427222, -0.0677778, 0.176111, ],
[0.426667, -0.07, 0.175, ],
[0.426111, -0.0722222, 0.173889, ],
[0.333333, -0.0766667, 0.186667, ],
[0.332778, -0.0788889, 0.185556, ],
[0.332222, -0.0811111, 0.184444, ],
[0.331667, -0.0833333, 0.183333, ],
[0.331111, -0.0855556, 0.182222, ],
[0.330556, -0.0877778, 0.181111, ],
[0.33, -0.09, 0.18, ],
[0.329444, -0.0922222, 0.178889, ],
[0.328889, -0.0944444, 0.177778, ],
[0.328333, -0.0966667, 0.176667, ],
[0.344444, -0.0744444, 0.186111, ],
[0.343889, -0.0766667, 0.185, ],
[0.343333, -0.0788889, 0.183889, ],
[0.342778, -0.0811111, 0.182778, ],
[0.342222, -0.0833333, 0.181667, ],
[0.341667, -0.0855556, 0.180556, ],
[0.341111, -0.0877778, 0.179444, ],
[0.340556, -0.09, 0.178333, ],
[0.34, -0.0922222, 0.177222, ],
[0.339444, -0.0944444, 0.176111, ],
[0.355556, -0.0722222, 0.185556, ],
[0.355, -0.0744444, 0.184444, ],
[0.354444, -0.0766667, 0.183333, ],
[0.353889, -0.0788889, 0.182222, ],
[0.353333, -0.0811111, 0.181111, ],
[0.352778, -0.0833333, 0.18, ],
[0.352222, -0.0855556, 0.178889, ],
[0.351667, -0.0877778, 0.177778, ],
[0.351111, -0.09, 0.176667, ],
[0.350556, -0.0922222, 0.175556, ],
[0.366667, -0.07, 0.185, ],
[0.366111, -0.0722222, 0.183889, ],
[0.365556, -0.0744444, 0.182778, ],
[0.365, -0.0766667, 0.181667, ],
[0.364444, -0.0788889, 0.180556, ],
[0.363889, -0.0811111, 0.179444, ],
[0.363333, -0.0833333, 0.178333, ],
[0.362778, -0.0855556, 0.177222, ],
[0.362222, -0.0877778, 0.176111, ],
[0.361667, -0.09, 0.175, ],
[0.377778, -0.0677778, 0.184444, ],
[0.377222, -0.07, 0.183333, ],
[0.376667, -0.0722222, 0.182222, ],
[0.376111, -0.0744444, 0.181111, ],
[0.375556, -0.0766667, 0.18, ],
[0.375, -0.0788889, 0.178889, ],
[0.374444, -0.0811111, 0.177778, ],
[0.373889, -0.0833333, 0.176667, ],
[0.373333, -0.0855556, 0.175556, ],
[0.372778, -0.0877778, 0.174444, ],
[0.388889, -0.0655556, 0.183889, ],
[0.388333, -0.0677778, 0.182778, ],
[0.387778, -0.07, 0.181667, ],
[0.387222, -0.0722222, 0.180556, ],
[0.386667, -0.0744444, 0.179444, ],
[0.386111, -0.0766667, 0.178333, ],
[0.385556, -0.0788889, 0.177222, ],
[0.385, -0.0811111, 0.176111, ],
[0.384444, -0.0833333, 0.175, ],
[0.383889, -0.0855556, 0.173889, ],
[0.4, -0.0633333, 0.183333, ],
[0.399444, -0.0655556, 0.182222, ],
[0.398889, -0.0677778, 0.181111, ],
[0.398333, -0.07, 0.18, ],
[0.397778, -0.0722222, 0.178889, ],
[0.397222, -0.0744444, 0.177778, ],
[0.396667, -0.0766667, 0.176667, ],
[0.396111, -0.0788889, 0.175556, ],
[0.395556, -0.0811111, 0.174444, ],
[0.395, -0.0833333, 0.173333, ],
[0.411111, -0.0611111, 0.182778, ],
[0.410556, -0.0633333, 0.181667, ],
[0.41, -0.0655556, 0.180556, ],
[0.409444, -0.0677778, 0.179444, ],
[0.408889, -0.07, 0.178333, ],
[0.408333, -0.0722222, 0.177222, ],
[0.407778, -0.0744444, 0.176111, ],
[0.407222, -0.0766667, 0.175, ],
[0.406667, -0.0788889, 0.173889, ],
[0.406111, -0.0811111, 0.172778, ],
[0.422222, -0.0588889, 0.182222, ],
[0.421667, -0.0611111, 0.181111, ],
[0.421111, -0.0633333, 0.18, ],
[0.420556, -0.0655556, 0.178889, ],
[0.42, -0.0677778, 0.177778, ],
[0.419444, -0.07, 0.176667, ],
[0.418889, -0.0722222, 0.175556, ],
[0.418333, -0.0744444, 0.174444, ],
[0.417778, -0.0766667, 0.173333, ],
[0.417222, -0.0788889, 0.172222, ],
[0.433333, -0.0566667, 0.181667, ],
[0.432778, -0.0588889, 0.180556, ],
[0.432222, -0.0611111, 0.179444, ],
[0.431667, -0.0633333, 0.178333, ],
[0.431111, -0.0655556, 0.177222, ],
[0.430556, -0.0677778, 0.176111, ],
[0.43, -0.07, 0.175, ],
[0.429444, -0.0722222, 0.173889, ],
[0.428889, -0.0744444, 0.172778, ],
[0.428333, -0.0766667, 0.171667, ],
[0.335556, -0.0811111, 0.184444, ],
[0.335, -0.0833333, 0.183333, ],
[0.334444, -0.0855556, 0.182222, ],
[0.333889, -0.0877778, 0.181111, ],
[0.333333, -0.09, 0.18, ],
[0.332778, -0.0922222, 0.178889, ],
[0.332222, -0.0944444, 0.177778, ],
[0.331667, -0.0966667, 0.176667, ],
[0.331111, -0.0988889, 0.175556, ],
[0.330556, -0.101111, 0.174444, ],
[0.346667, -0.0788889, 0.183889, ],
[0.346111, -0.0811111, 0.182778, ],
[0.345556, -0.0833333, 0.181667, ],
[0.345, -0.0855556, 0.180556, ],
[0.344444, -0.0877778, 0.179444, ],
[0.343889, -0.09, 0.178333, ],
[0.343333, -0.0922222, 0.177222, ],
[0.342778, -0.0944444, 0.176111, ],
[0.342222, -0.0966667, 0.175, ],
[0.341667, -0.0988889, 0.173889, ],
[0.357778, -0.0766667, 0.183333, ],
[0.357222, -0.0788889, 0.182222, ],
[0.356667, -0.0811111, 0.181111, ],
[0.356111, -0.0833333, 0.18, ],
[0.355556, -0.0855556, 0.178889, ],
[0.355, -0.0877778, 0.177778, ],
[0.354444, -0.09, 0.176667, ],
[0.353889, -0.0922222, 0.175556, ],
[0.353333, -0.0944444, 0.174444, ],
[0.352778, -0.0966667, 0.173333, ],
[0.368889, -0.0744444, 0.182778, ],
[0.368333, -0.0766667, 0.181667, ],
[0.367778, -0.0788889, 0.180556, ],
[0.367222, -0.0811111, 0.179444, ],
[0.366667, -0.0833333, 0.178333, ],
[0.366111, -0.0855556, 0.177222, ],
[0.365556, -0.0877778, 0.176111, ],
[0.365, -0.09, 0.175, ],
[0.364444, -0.0922222, 0.173889, ],
[0.363889, -0.0944444, 0.172778, ],
[0.38, -0.0722222, 0.182222, ],
[0.379444, -0.0744444, 0.181111, ],
[0.378889, -0.0766667, 0.18, ],
[0.378333, -0.0788889, 0.178889, ],
[0.377778, -0.0811111, 0.177778, ],
[0.377222, -0.0833333, 0.176667, ],
[0.376667, -0.0855556, 0.175556, ],
[0.376111, -0.0877778, 0.174444, ],
[0.375556, -0.09, 0.173333, ],
[0.375, -0.0922222, 0.172222, ],
[0.391111, -0.07, 0.181667, ],
[0.390556, -0.0722222, 0.180556, ],
[0.39, -0.0744444, 0.179444, ],
[0.389444, -0.0766667, 0.178333, ],
[0.388889, -0.0788889, 0.177222, ],
[0.388333, -0.0811111, 0.176111, ],
[0.387778, -0.0833333, 0.175, ],
[0.387222, -0.0855556, 0.173889, ],
[0.386667, -0.0877778, 0.172778, ],
[0.386111, -0.09, 0.171667, ],
[0.402222, -0.0677778, 0.181111, ],
[0.401667, -0.07, 0.18, ],
[0.401111, -0.0722222, 0.178889, ],
[0.400556, -0.0744444, 0.177778, ],
[0.4, -0.0766667, 0.176667, ],
[0.399444, -0.0788889, 0.175556, ],
[0.398889, -0.0811111, 0.174444, ],
[0.398333, -0.0833333, 0.173333, ],
[0.397778, -0.0855556, 0.172222, ],
[0.397222, -0.0877778, 0.171111, ],
[0.413333, -0.0655556, 0.180556, ],
[0.412778, -0.0677778, 0.179444, ],
[0.412222, -0.07, 0.178333, ],
[0.411667, -0.0722222, 0.177222, ],
[0.411111, -0.0744444, 0.176111, ],
[0.410556, -0.0766667, 0.175, ],
[0.41, -0.0788889, 0.173889, ],
[0.409444, -0.0811111, 0.172778, ],
[0.408889, -0.0833333, 0.171667, ],
[0.408333, -0.0855556, 0.170556, ],
[0.424444, -0.0633333, 0.18, ],
[0.423889, -0.0655556, 0.178889, ],
[0.423333, -0.0677778, 0.177778, ],
[0.422778, -0.07, 0.176667, ],
[0.422222, -0.0722222, 0.175556, ],
[0.421667, -0.0744444, 0.174444, ],
[0.421111, -0.0766667, 0.173333, ],
[0.420556, -0.0788889, 0.172222, ],
[0.42, -0.0811111, 0.171111, ],
[0.419444, -0.0833333, 0.17, ],
[0.435556, -0.0611111, 0.179444, ],
[0.435, -0.0633333, 0.178333, ],
[0.434444, -0.0655556, 0.177222, ],
[0.433889, -0.0677778, 0.176111, ],
[0.433333, -0.07, 0.175, ],
[0.432778, -0.0722222, 0.173889, ],
[0.432222, -0.0744444, 0.172778, ],
[0.431667, -0.0766667, 0.171667, ],
[0.431111, -0.0788889, 0.170556, ],
[0.430556, -0.0811111, 0.169444, ],
[0.337778, -0.0855556, 0.182222, ],
[0.337222, -0.0877778, 0.181111, ],
[0.336667, -0.09, 0.18, ],
[0.336111, -0.0922222, 0.178889, ],
[0.335556, -0.0944444, 0.177778, ],
[0.335, -0.0966667, 0.176667, ],
[0.334444, -0.0988889, 0.175556, ],
[0.333889, -0.101111, 0.174444, ],
[0.333333, -0.103333, 0.173333, ],
[0.332778, -0.105556, 0.172222, ],
[0.348889, -0.0833333, 0.181667, ],
[0.348333, -0.0855556, 0.180556, ],
[0.347778, -0.0877778, 0.179444, ],
[0.347222, -0.09, 0.178333, ],
[0.346667, -0.0922222, 0.177222, ],
[0.346111, -0.0944444, 0.176111, ],
[0.345556, -0.0966667, 0.175, ],
[0.345, -0.0988889, 0.173889, ],
[0.344444, -0.101111, 0.172778, ],
[0.343889, -0.103333, 0.171667, ],
[0.36, -0.0811111, 0.181111, ],
[0.359444, -0.0833333, 0.18, ],
[0.358889, -0.0855556, 0.178889, ],
[0.358333, -0.0877778, 0.177778, ],
[0.357778, -0.09, 0.176667, ],
[0.357222, -0.0922222, 0.175556, ],
[0.356667, -0.0944444, 0.174444, ],
[0.356111, -0.0966667, 0.173333, ],
[0.355556, -0.0988889, 0.172222, ],
[0.355, -0.101111, 0.171111, ],
[0.371111, -0.0788889, 0.180556, ],
[0.370556, -0.0811111, 0.179444, ],
[0.37, -0.0833333, 0.178333, ],
[0.369444, -0.0855556, 0.177222, ],
[0.368889, -0.0877778, 0.176111, ],
[0.368333, -0.09, 0.175, ],
[0.367778, -0.0922222, 0.173889, ],
[0.367222, -0.0944444, 0.172778, ],
[0.366667, -0.0966667, 0.171667, ],
[0.366111, -0.0988889, 0.170556, ],
[0.382222, -0.0766667, 0.18, ],
[0.381667, -0.0788889, 0.178889, ],
[0.381111, -0.0811111, 0.177778, ],
[0.380556, -0.0833333, 0.176667, ],
[0.38, -0.0855556, 0.175556, ],
[0.379444, -0.0877778, 0.174444, ],
[0.378889, -0.09, 0.173333, ],
[0.378333, -0.0922222, 0.172222, ],
[0.377778, -0.0944444, 0.171111, ],
[0.377222, -0.0966667, 0.17, ],
[0.393333, -0.0744444, 0.179444, ],
[0.392778, -0.0766667, 0.178333, ],
[0.392222, -0.0788889, 0.177222, ],
[0.391667, -0.0811111, 0.176111, ],
[0.391111, -0.0833333, 0.175, ],
[0.390556, -0.0855556, 0.173889, ],
[0.39, -0.0877778, 0.172778, ],
[0.389444, -0.09, 0.171667, ],
[0.388889, -0.0922222, 0.170556, ],
[0.388333, -0.0944444, 0.169444, ],
[0.404444, -0.0722222, 0.178889, ],
[0.403889, -0.0744444, 0.177778, ],
[0.403333, -0.0766667, 0.176667, ],
[0.402778, -0.0788889, 0.175556, ],
[0.402222, -0.0811111, 0.174444, ],
[0.401667, -0.0833333, 0.173333, ],
[0.401111, -0.0855556, 0.172222, ],
[0.400556, -0.0877778, 0.171111, ],
[0.4, -0.09, 0.17, ],
[0.399444, -0.0922222, 0.168889, ],
[0.415556, -0.07, 0.178333, ],
[0.415, -0.0722222, 0.177222, ],
[0.414444, -0.0744444, 0.176111, ],
[0.413889, -0.0766667, 0.175, ],
[0.413333, -0.0788889, 0.173889, ],
[0.412778, -0.0811111, 0.172778, ],
[0.412222, -0.0833333, 0.171667, ],
[0.411667, -0.0855556, 0.170556, ],
[0.411111, -0.0877778, 0.169444, ],
[0.410556, -0.09, 0.168333, ],
[0.426667, -0.0677778, 0.177778, ],
[0.426111, -0.07, 0.176667, ],
[0.425556, -0.0722222, 0.175556, ],
[0.425, -0.0744444, 0.174444, ],
[0.424444, -0.0766667, 0.173333, ],
[0.423889, -0.0788889, 0.172222, ],
[0.423333, -0.0811111, 0.171111, ],
[0.422778, -0.0833333, 0.17, ],
[0.422222, -0.0855556, 0.168889, ],
[0.421667, -0.0877778, 0.167778, ],
[0.437778, -0.0655556, 0.177222, ],
[0.437222, -0.0677778, 0.176111, ],
[0.436667, -0.07, 0.175, ],
[0.436111, -0.0722222, 0.173889, ],
[0.435556, -0.0744444, 0.172778, ],
[0.435, -0.0766667, 0.171667, ],
[0.434444, -0.0788889, 0.170556, ],
[0.433889, -0.0811111, 0.169444, ],
[0.433333, -0.0833333, 0.168333, ],
[0.432778, -0.0855556, 0.167222, ],
[0.34, -0.09, 0.18, ],
[0.339444, -0.0922222, 0.178889, ],
[0.338889, -0.0944444, 0.177778, ],
[0.338333, -0.0966667, 0.176667, ],
[0.337778, -0.0988889, 0.175556, ],
[0.337222, -0.101111, 0.174444, ],
[0.336667, -0.103333, 0.173333, ],
[0.336111, -0.105556, 0.172222, ],
[0.335556, -0.107778, 0.171111, ],
[0.335, -0.11, 0.17, ],
[0.351111, -0.0877778, 0.179444, ],
[0.350556, -0.09, 0.178333, ],
[0.35, -0.0922222, 0.177222, ],
[0.349444, -0.0944444, 0.176111, ],
[0.348889, -0.0966667, 0.175, ],
[0.348333, -0.0988889, 0.173889, ],
[0.347778, -0.101111, 0.172778, ],
[0.347222, -0.103333, 0.171667, ],
[0.346667, -0.105556, 0.170556, ],
[0.346111, -0.107778, 0.169444, ],
[0.362222, -0.0855556, 0.178889, ],
[0.361667, -0.0877778, 0.177778, ],
[0.361111, -0.09, 0.176667, ],
[0.360556, -0.0922222, 0.175556, ],
[0.36, -0.0944444, 0.174444, ],
[0.359444, -0.0966667, 0.173333, ],
[0.358889, -0.0988889, 0.172222, ],
[0.358333, -0.101111, 0.171111, ],
[0.357778, -0.103333, 0.17, ],
[0.357222, -0.105556, 0.168889, ],
[0.373333, -0.0833333, 0.178333, ],
[0.372778, -0.0855556, 0.177222, ],
[0.372222, -0.0877778, 0.176111, ],
[0.371667, -0.09, 0.175, ],
[0.371111, -0.0922222, 0.173889, ],
[0.370556, -0.0944444, 0.172778, ],
[0.37, -0.0966667, 0.171667, ],
[0.369444, -0.0988889, 0.170556, ],
[0.368889, -0.101111, 0.169444, ],
[0.368333, -0.103333, 0.168333, ],
[0.384444, -0.0811111, 0.177778, ],
[0.383889, -0.0833333, 0.176667, ],
[0.383333, -0.0855556, 0.175556, ],
[0.382778, -0.0877778, 0.174444, ],
[0.382222, -0.09, 0.173333, ],
[0.381667, -0.0922222, 0.172222, ],
[0.381111, -0.0944444, 0.171111, ],
[0.380556, -0.0966667, 0.17, ],
[0.38, -0.0988889, 0.168889, ],
[0.379444, -0.101111, 0.167778, ],
[0.395556, -0.0788889, 0.177222, ],
[0.395, -0.0811111, 0.176111, ],
[0.394444, -0.0833333, 0.175, ],
[0.393889, -0.0855556, 0.173889, ],
[0.393333, -0.0877778, 0.172778, ],
[0.392778, -0.09, 0.171667, ],
[0.392222, -0.0922222, 0.170556, ],
[0.391667, -0.0944444, 0.169444, ],
[0.391111, -0.0966667, 0.168333, ],
[0.390556, -0.0988889, 0.167222, ],
[0.406667, -0.0766667, 0.176667, ],
[0.406111, -0.0788889, 0.175556, ],
[0.405556, -0.0811111, 0.174444, ],
[0.405, -0.0833333, 0.173333, ],
[0.404444, -0.0855556, 0.172222, ],
[0.403889, -0.0877778, 0.171111, ],
[0.403333, -0.09, 0.17, ],
[0.402778, -0.0922222, 0.168889, ],
[0.402222, -0.0944444, 0.167778, ],
[0.401667, -0.0966667, 0.166667, ],
[0.417778, -0.0744444, 0.176111, ],
[0.417222, -0.0766667, 0.175, ],
[0.416667, -0.0788889, 0.173889, ],
[0.416111, -0.0811111, 0.172778, ],
[0.415556, -0.0833333, 0.171667, ],
[0.415, -0.0855556, 0.170556, ],
[0.414444, -0.0877778, 0.169444, ],
[0.413889, -0.09, 0.168333, ],
[0.413333, -0.0922222, 0.167222, ],
[0.412778, -0.0944444, 0.166111, ],
[0.428889, -0.0722222, 0.175556, ],
[0.428333, -0.0744444, 0.174444, ],
[0.427778, -0.0766667, 0.173333, ],
[0.427222, -0.0788889, 0.172222, ],
[0.426667, -0.0811111, 0.171111, ],
[0.426111, -0.0833333, 0.17, ],
[0.425556, -0.0855556, 0.168889, ],
[0.425, -0.0877778, 0.167778, ],
[0.424444, -0.09, 0.166667, ],
[0.423889, -0.0922222, 0.165556, ],
[0.44, -0.07, 0.175, ],
[0.439444, -0.0722222, 0.173889, ],
[0.438889, -0.0744444, 0.172778, ],
[0.438333, -0.0766667, 0.171667, ],
[0.437778, -0.0788889, 0.170556, ],
[0.437222, -0.0811111, 0.169444, ],
[0.436667, -0.0833333, 0.168333, ],
[0.436111, -0.0855556, 0.167222, ],
[0.435556, -0.0877778, 0.166111, ],
[0.435, -0.09, 0.165, ],
])


In [ ]:
np.random.seed(123)
pts = np.hstack([micro_us + dns_data[0], dns_data[0]])

isubset = np.random.choice(pts.shape[0], 100, replace=False)
subset = pts[isubset]

spts = np.array([[s[:3], s[3:]] for s in subset])
print(spts.shape)

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(111, projection='3d')
u = dict((key, np.zeros(3)) for key in nodes.keys())
plot_hex(10, nodes, conn, u, ax)
us = np.array([[0.379335, -0.0706384, 0.181941],
[0.42863, -0.0607796, 0.179477],
[0.438488, -0.0804972, 0.169618],
[0.389194, -0.090356, 0.172083],
[0.376871, -0.0804972, 0.177012],
[0.426165, -0.0706384, 0.174547],
[0.436024, -0.090356, 0.164688],
[0.38673, -0.100215, 0.167153],
])
u = dict([(key, us[i]) for key, i in zip(nodes.keys(), range(8))])
plot_hex(10, nodes, conn, u, ax)

ax.scatter(*zip(*dns_data[0][isubset]), color='b')
ax.scatter(*zip(*dns_data[.1][isubset]), color='r')

for sp in spts:
    
    ax.plot(sp.T[0], sp.T[1], sp.T[2], color='k')
    

plt.show()


(100, 2, 3)
